In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Open the Google Sheet using its name
sheet = gc.open('ET Form 1 & 2 - INSP-07-FM-07 Rev08').worksheet('Form One (1)')
sheet1 = gc.open('ET Form 1 & 2 - INSP-07-FM-07 Rev08').worksheet('Form Two (2)')

# **df**

In [ ]:
df = get_as_dataframe(sheet, evaluate_formulas=True, headers=True)

# Drop the headers and reset the index
df = df.iloc[1:].reset_index(drop=True)

# Rename columns with custom header names
custom_headers = ['Added Time', 'IP Address', 'Done By', 'Lot No.', 'Tool Number (S. No.)', 'Kevin Probe Test', 'For 4Wire', 'DateCode', 'Job Type', 'Machine',
                  'X-Cut Allowed', 'IPC Class', 'Mass Lam', 'Resistive', 'Continuity', 'Test Voltage', 'Isolation', 'Adjacency Used ( Emma Only)', 'Mfg Qty', 'Tested Qty',
                  '1st Passed Qty', 'Open', 'Short', 'SDP', 'Lot ID Suffix', 'Full Lot No', '1st Passed Qty 1', 'Tested Qty 1']

df.columns = custom_headers

df = df.dropna(subset=['Added Time'])
df

,Added Time,IP Address,Done By,Lot No.,Tool Number (S. No.),Kevin Probe Test,For 4Wire,DateCode,Job Type,Machine,...,Mfg Qty,Tested Qty,1st Passed Qty,Open,Short,SDP,Lot ID Suffix,Full Lot No,1st Passed Qty 1,Tested Qty 1
0,01-Jan-2023 1:30:09,165.225.230.205,102480,11387972,S03804_A,No,NaN,2252,Standard,Versa,...,630,592,465,63,64,38,NaN,11387972,465,592
1,01-Jul-2023 0:38:13,165.225.117.70,101754,12105804,S11664_E,No,NaN,2623,QTA,KAIMA,...,120,91,91,0,0,29,NaN,12105804,91,91
2,01-Jul-2023 3:19:06,165.225.116.126,102477,12107143,S12881_B,No,NaN,2623,NPI,6151 (1),...,48,42,42,0,0,6,NaN,12107143,42,42
3,01-Jul-2023 3:21:07,165.225.116.126,102477,12107226,S12949_A,No,NaN,2523,QTA,4033,...,336,326,322,4,0,10,NaN,12107226,322,326
4,01-Jul-2023 5:13:25,165.225.117.70,101754,12073207,S11515_J,No,NaN,2623,Standard,KAIMA,...,432,376,336,40,0,56,NaN,12073207,336,376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12568,08-Jan-2025 13:03:24,165.225.113.53,200090,12587705,S13203_A,No,NaN,225,Standard,KAIMA,...,2760,2755,2753,1,1,5,NaN,12587705,2753,2755
12569,08-Jan-2025 14:45:20,165.225.113.53,200090,12587392,S12453_A,No,NaN,2452,Standard,KAIMA,...,540,513,510,1,2,27,(Silver),12587392(Silver),510,513
12570,08-Jan-2025 16:12:20,165.225.230.186,101664,12588590,S12695_B,No,NaN,225,QTA,6151 (2),...,120,89,88,1,0,31,(Silver),12588590(Silver),88,89
12571,08-Jan-2025 16:14:55,165.225.113.53,102415,12591568,S09018_C,No,NaN,225,Standard,LM1,...,84,79,78,1,0,5,NaN,12591568,78,79


In [ ]:
cols_to_drop = ['IP Address', '1st Passed Qty 1', 'Tested Qty 1']

# Drop the specified columns
df = df.drop(columns=cols_to_drop)

df.head()

,Added Time,Done By,Lot No.,Tool Number (S. No.),Kevin Probe Test,For 4Wire,DateCode,Job Type,Machine,X-Cut Allowed,...,Isolation,Adjacency Used ( Emma Only),Mfg Qty,Tested Qty,1st Passed Qty,Open,Short,SDP,Lot ID Suffix,Full Lot No
0,01-Jan-2023 1:30:09,102480,11387972,S03804_A,No,NaN,2252,Standard,Versa,3.0,...,10.0,No,630,592,465,63,64,38,NaN,11387972
1,01-Jul-2023 0:38:13,101754,12105804,S11664_E,No,NaN,2623,QTA,KAIMA,0.0,...,10.0,No,120,91,91,0,0,29,NaN,12105804
2,01-Jul-2023 3:19:06,102477,12107143,S12881_B,No,NaN,2623,NPI,6151 (1),1.0,...,1048.0,Yes,48,42,42,0,0,6,NaN,12107143
3,01-Jul-2023 3:21:07,102477,12107226,S12949_A,No,NaN,2523,QTA,4033,0.0,...,10.0,Yes,336,326,322,4,0,10,NaN,12107226
4,01-Jul-2023 5:13:25,101754,12073207,S11515_J,No,NaN,2623,Standard,KAIMA,0.0,...,10.0,No,432,376,336,40,0,56,NaN,12073207


In [ ]:
# Convert all string columns to uppercase
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

<ipython-input-7-59832e7dc377>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [ ]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Display missing values for each column
if missing_values.any():
    print("Missing Values in Each Column:")
    print(missing_values[missing_values > 0])
else:
    print("There are no missing values in the dataset.")

Missing Values in Each Column:
For 4Wire        11566
Open                 6
Short                6
SDP                  6
Lot ID Suffix    10415
dtype: int64


In [ ]:
df['Added Time'] = df['Added Time'].astype(str)

def custom_parse_date(date_str):
    # Try parsing with the first format
    try:
        return pd.to_datetime(date_str, format="%d-%b-%Y %H:%M:%S")
    except ValueError:
        # If it fails, try the second format
        try:
            return pd.to_datetime(date_str, format="%m/%d/%Y")
        except ValueError:
            return pd.NaT  # Return NaT for unrecognized formats

# Convert 'Added Time' to datetime using the custom parser
df['Added Time'] = df['Added Time'].astype(str).apply(custom_parse_date)

# Check for NaT values and log them
problematic_dates = df[df['Added Time'].isna()]['Added Time']
if not problematic_dates.empty:
    print("Problematic Dates:")
    print(problematic_dates)

# Standardize the 'Standardized Date' column, formatting only valid dates
# df['Standardized Date'] = df['Added Time'].apply(lambda x: x.strftime('%d-%m-%Y') if pd.notna(x) else pd.NaT)

# Ensure 'Standardized Date' is a copy of 'Added Time' in datetime format (no formatting yet)
df['Standardized Date'] = df['Added Time']

# Convert 'Standardized Date' to remove the time component but keep as datetime64
df['Standardized Date'] = pd.to_datetime(df['Standardized Date']).dt.normalize()

# Check for NaT values
num_nat = df['Added Time'].isna().sum()
print(f"Number of NaT values after custom parsing: {num_nat}")

# Display the DataFrame
print(df[['Added Time', 'Standardized Date']].head(20))

Number of NaT values after custom parsing: 0
            Added Time Standardized Date
0  2023-01-01 01:30:09        2023-01-01
1  2023-07-01 00:38:13        2023-07-01
2  2023-07-01 03:19:06        2023-07-01
3  2023-07-01 03:21:07        2023-07-01
4  2023-07-01 05:13:25        2023-07-01
5  2023-07-01 06:12:54        2023-07-01
6  2023-07-01 06:17:43        2023-07-01
7  2023-07-01 06:33:55        2023-07-01
8  2023-07-01 09:30:48        2023-07-01
9  2023-07-01 09:34:53        2023-07-01
10 2023-07-01 10:11:23        2023-07-01
11 2023-07-01 10:15:46        2023-07-01
12 2023-07-01 11:56:39        2023-07-01
13 2023-07-01 14:05:59        2023-07-01
14 2023-07-01 14:27:13        2023-07-01
15 2023-07-01 16:16:20        2023-07-01
16 2023-07-01 16:20:31        2023-07-01
17 2023-07-01 16:23:02        2023-07-01
18 2023-07-01 16:40:14        2023-07-01
19 2023-07-01 16:44:32        2023-07-01


In [ ]:
print(df['Added Time'].isna().sum())  # Check for NaN values in 'Added Time'
print(df['Standardized Date'].isna().sum())  # Check for NaT values in 'Standardized Date'

0
0


In [ ]:
# Check data types of each column
print(df.dtypes)

Added Time                     datetime64[ns]
Done By                                object
Lot No.                                object
Tool Number (S. No.)                   object
Kevin Probe Test                       object
For 4Wire                              object
DateCode                               object
Job Type                               object
Machine                                object
X-Cut Allowed                         float64
IPC Class                             float64
Mass Lam                               object
Resistive                              object
Continuity                            float64
Test Voltage                          float64
Isolation                             float64
Adjacency Used ( Emma Only)            object
Mfg Qty                                 int64
Tested Qty                              int64
1st Passed Qty                          int64
Open                                  float64
Short                             

In [ ]:
# List of columns where you want to replace NaN values with 0
columns_to_fill = ['Open', 'Short', 'SDP']

# Replace NaN values with 0 in the selected columns
df[columns_to_fill] = df[columns_to_fill].fillna(0).astype(int)

In [ ]:
# Check for missing values in the DataFrame
missing_values = df.isnull().sum()

# Display missing values for each column
if missing_values.any():
    print("Missing Values in Each Column:")
    print(missing_values[missing_values > 0])
else:
    print("There are no missing values in the dataset.")

Missing Values in Each Column:
For 4Wire        11566
Lot ID Suffix    10415
dtype: int64


In [ ]:
# Convert to numeric and check which values fail conversion (non-numeric values will be set to NaN)
isolation_check = pd.to_numeric(df['Isolation'], errors='coerce')

# Identify rows where conversion failed (non-numeric values or NaN values)
problematic_isolation_values = df.loc[isolation_check.isna() & df['Isolation'].notna(), 'Isolation']

# Display the problematic values
print("Problematic values preventing conversion to int:")
print(problematic_isolation_values)

Problematic values preventing conversion to int:
Series([], Name: Isolation, dtype: float64)


In [ ]:
df['Done By'] = df['Done By'].astype(str)
df['Lot No.'] = df['Lot No.'].astype(str)
df['Tool Number (S. No.)'] = df['Tool Number (S. No.)'].astype(str)
df['DateCode'] = df['DateCode'].astype(str)
df['X-Cut Allowed'] = df['X-Cut Allowed'].astype('object')
df['IPC Class'] = df['IPC Class'].astype('object')
df['Continuity'] = df['Continuity'].astype('int', errors='ignore')
df['Test Voltage'] = df['Test Voltage'].astype('int', errors='ignore')
df['Isolation'] = df['Isolation'].astype('int', errors='ignore')
df['Mfg Qty'] = df['Mfg Qty'].astype('int', errors='ignore')
df['Tested Qty'] = df['Tested Qty'].astype('int', errors='ignore')
df['1st Passed Qty'] = df['1st Passed Qty'].astype('int', errors='ignore')
df.rename(columns={'Full Lot No': 'Full Lot ID'}, inplace=True)
print(df.dtypes)

Added Time                     datetime64[ns]
Done By                                object
Lot No.                                object
Tool Number (S. No.)                   object
Kevin Probe Test                       object
For 4Wire                              object
DateCode                               object
Job Type                               object
Machine                                object
X-Cut Allowed                          object
IPC Class                              object
Mass Lam                               object
Resistive                              object
Continuity                              int64
Test Voltage                            int64
Isolation                               int64
Adjacency Used ( Emma Only)            object
Mfg Qty                                 int64
Tested Qty                              int64
1st Passed Qty                          int64
Open                                    int64
Short                             

In [ ]:
# Check if any Tool Number does not have 8 characters
invalid_entries = df[df['Tool Number (S. No.)'].str.len() != 8]

# Display the invalid entries with their row indices
if not invalid_entries.empty:
    print("Invalid Tool Numbers and their Row Indices:")
    for index, row in invalid_entries.iterrows():
        print(f"Row {index}: {row['Tool Number (S. No.)']}")
else:
    print("All Tool Numbers are valid.")

All Tool Numbers are valid.


In [ ]:
# Define the regex pattern
pattern = r'^S\d{5}_[A-Z]$'

# Check if any Tool Number does not match the pattern
invalid_entries = df[~df['Tool Number (S. No.)'].str.match(pattern, na=False)]

print("Invalid Tool Numbers:", invalid_entries[['Tool Number (S. No.)']])

Invalid Tool Numbers:       Tool Number (S. No.)
307               S04593-D
530               S07957-A
1126              S11607-F
2032              S06592-B
2453              S13122-A
2840              S08365-D
2881              S12305-C
3481              S03878-C
3588              S11564-E
3638              S12935-D
3682              S08315-A
4394              S05768-C
5071              S07037-B
6755              S13033-A
6897              S10799-H
7076              S05809+C
7616              S09888-A
7896              S12453-A
7963              S10854-D
8819              S12305-C
8887              S04655-B
10916             S07930-E
11098             S09504-F
11174             S10157-F
11584             S08274-C
11820             A11834_A
12086             A13117_B
12162             S13387-B
12244             S1197501


In [ ]:
# Replace '-' and '+' with '_' in the 'Tool Number (S. No.)' column
df['Tool Number (S. No.)'] = df['Tool Number (S. No.)'].str.replace('[-+]', '_', regex=True)

In [ ]:
# Define the regex pattern
pattern = r'^S\d{5}_[A-Z]$'

# Check if any Tool Number does not match the pattern
invalid_entries = df[~df['Tool Number (S. No.)'].str.match(pattern, na=False)]

print("Invalid Tool Numbers:", invalid_entries[['Tool Number (S. No.)']])

Invalid Tool Numbers:       Tool Number (S. No.)
11820             A11834_A
12086             A13117_B
12244             S1197501


In [ ]:
# Define a function to split the 'Lot No.' into 'Split ID' and 'Lot ID'
def split_lot_no(lot_no):
    parts = lot_no.split(', ')
    if len(parts) == 2:
        return parts[0], parts[1]  # Return Split ID and Lot ID
    else:
        return np.NaN, parts[0]  # No Split ID, return None and the whole Lot ID

# Apply the function to the 'Lot No.' column
df[['Split ID', 'Lot ID']] = df['Lot No.'].apply(split_lot_no).apply(pd.Series)

print(df.loc[3918])

Added Time                     2023-12-12 00:00:00
Done By                                     200090
Lot No.                                A, 12215287
Tool Number (S. No.)                      S09799_E
Kevin Probe Test                                NO
For 4Wire                                      NaN
DateCode                                      5023
Job Type                                  STANDARD
Machine                                      KAIMA
X-Cut Allowed                                  3.0
IPC Class                                      2.0
Mass Lam                                        NO
Resistive                                      YES
Continuity                                      20
Test Voltage                                   100
Isolation                                       10
Adjacency Used ( Emma Only)                     NO
Mfg Qty                                       2080
Tested Qty                                    2080
1st Passed Qty                 

In [ ]:
# Identify rows with invalid Date Codes (not 4 characters long), but accept None values
invalid_date_codes = df[(df['DateCode'].notna()) & (df['DateCode'] != "NO DATE CODE AVAILABLE") & (df['DateCode'].str.len() != 4)]

print("Invalid Date Codes:", invalid_date_codes[['DateCode']])

Invalid Date Codes:       DateCode
42         623
75         723
167        623
168        623
190        723
...        ...
12567      125
12568      225
12570      225
12571      225
12572      125

[1007 rows x 1 columns]


In [ ]:
# Identify rows where the DateCode is exactly 3 characters long
invalid_date_codes = df[
    (df['DateCode'].notna()) &
    (df['DateCode'] != "NO DATE CODE AVAILABLE") &
    (df['DateCode'].str.len() == 3)
]

# Add '0' to the start of the 3-character DateCodes
df.loc[invalid_date_codes.index, 'DateCode'] = df.loc[invalid_date_codes.index, 'DateCode'].apply(lambda x: '0' + x)

# Print the updated DataFrame with modified DateCodes
print("Updated rows with 3-character Date Codes (now padded with '0'):")
print(df[['DateCode']].loc[invalid_date_codes.index])


Updated rows with 3-character Date Codes (now padded with '0'):
      DateCode
42        0623
75        0723
167       0623
168       0623
190       0723
...        ...
12567     0125
12568     0225
12570     0225
12571     0225
12572     0125

[1007 rows x 1 columns]


In [ ]:
# Identify rows with invalid Date Codes (not 4 characters long), but accept None values
invalid_date_codes = df[(df['DateCode'].notna()) & (df['DateCode'] != "NO DATE CODE AVAILABLE") & (df['DateCode'].str.len() != 4)]

print("Invalid Date Codes:", invalid_date_codes[['DateCode']])

Invalid Date Codes: Empty DataFrame
Columns: [DateCode]
Index: []


In [ ]:
invalid_date_codes = df[
    (df['DateCode'].notna()) &
    (df['DateCode'] != "NO DATE CODE AVAILABLE") &
    (df['DateCode'].str.len() == 4) &
    (~df['DateCode'].str[:2].isin(['22', '23', '24'])) &  # First two characters not 22, 23, or 24
    (~df['DateCode'].str[-2:].isin(['22', '23', '24']))   # Last two characters not 22, 23, or 24
]

# Print the rows with 4-character DateCodes that do not contain '22', '23', or '24'
print("Rows with 4-character Date Codes that do not contain '22', '23', or '24' as the first two or last two characters:")
print(invalid_date_codes[['DateCode', 'Lot ID', 'Tool Number (S. No.)', 'Full Lot ID']])

Rows with 4-character Date Codes that do not contain '22', '23', or '24' as the first two or last two characters:
      DateCode    Lot ID Tool Number (S. No.)       Full Lot ID
11117     4254  12556409             S13575_A       A, 12556409
11131     4254  12556409             S13575_A          12556409
11787     4925  12576315             S11629_B  12576315(SILVER)
12032     5034  12571532             S12580_A          12571532
12356     0152  12583877             S12731_A          12583877
...        ...       ...                  ...               ...
12567     0125  12596873             S12962_A          12596873
12568     0225  12587705             S13203_A          12587705
12570     0225  12588590             S12695_B  12588590(SILVER)
12571     0225  12591568             S09018_C          12591568
12572     0125  12598625             S13202_A          12598625

[101 rows x 4 columns]


In [ ]:
# Create 'Date Code' and 'Date Code Input' columns based on the conditions
df['Date Code'] = df['DateCode'].apply(lambda x: 'Yes' if x not in ['NO DATE CODE AVAILABLE'] else 'No')
df['Date Code Input'] = df['DateCode'].apply(lambda x: x if x not in ['NO DATE CODE AVAILABLE'] else '')

df.loc[5]

,5
Added Time,2023-07-01 06:12:54
Done By,102817
Lot No.,12106732
Tool Number (S. No.),S11048_B
Kevin Probe Test,NO
For 4Wire,NaN
DateCode,NO DATE CODE AVAILABLE
Job Type,STANDARD
Machine,8161 (2)
X-Cut Allowed,0.0


In [ ]:
# Replace '/' with ',' in the specified column
df['Done By'] = df['Done By'].str.replace(r'\s*/\s*', ',', regex=True)

df.loc[9441]

,9441
Added Time,2024-08-24 18:00:08
Done By,"102878,102817,100140"
Lot No.,12452199
Tool Number (S. No.),S13156_A
Kevin Probe Test,NO
For 4Wire,NaN
DateCode,3324
Job Type,STANDARD
Machine,4033
X-Cut Allowed,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12573 entries, 0 to 12572
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Added Time                   12573 non-null  datetime64[ns]
 1   Done By                      12573 non-null  object        
 2   Lot No.                      12573 non-null  object        
 3   Tool Number (S. No.)         12573 non-null  object        
 4   Kevin Probe Test             12573 non-null  object        
 5   For 4Wire                    1007 non-null   object        
 6   DateCode                     12573 non-null  object        
 7   Job Type                     12573 non-null  object        
 8   Machine                      12573 non-null  object        
 9   X-Cut Allowed                12573 non-null  object        
 10  IPC Class                    12573 non-null  object        
 11  Mass Lam                     12573 non-null  o

In [ ]:
# # Function to check for invalid Employee IDs and print them with their row index
# def print_invalid_employee_ids(employee_id, index):
#     # Check if the length of the entire string is not equal to 6 characters
#     if len(employee_id.strip()) != 6:
#         print(f"Row {index}: Invalid Employee ID: {employee_id.strip()}")

# # Apply the function to the 'Done By' column, passing the index as well
# df['Done By'].apply(lambda x: print_invalid_employee_ids(x, df[df['Done By'] == x].index[0]))

In [ ]:
# Function to check for invalid Employee IDs and print them with their row index
def print_invalid_employee_ids(employee_ids, index):
    ids = employee_ids.split(',')  # Split the Employee IDs by comma
    invalid_ids = [eid.strip() for eid in ids if len(eid.strip()) != 6]  # Collect IDs that are not 6 characters

    if invalid_ids:  # If there are invalid IDs, print them with the row index
        print(f"Row {index}: Invalid Employee IDs: {invalid_ids}")

# Apply the function to the 'Employee ID' column, passing the index as well
df['Done By'].apply(lambda x: print_invalid_employee_ids(x, df[df['Done By'] == x].index[0]))

Row 107: Invalid Employee IDs: ['1020477']
Row 341: Invalid Employee IDs: ['1020878']
Row 1314: Invalid Employee IDs: ['2878']
Row 1639: Invalid Employee IDs: ['10247']
Row 1787: Invalid Employee IDs: ['1034052']
Row 2776: Invalid Employee IDs: ['103546 102415']
Row 2897: Invalid Employee IDs: ['10015']
Row 3713: Invalid Employee IDs: ['103.472']
Row 3752: Invalid Employee IDs: ['2000090']
Row 107: Invalid Employee IDs: ['1020477']
Row 4636: Invalid Employee IDs: ['1013546']
Row 4677: Invalid Employee IDs: ['12477']
Row 6347: Invalid Employee IDs: ['1002878']
Row 6813: Invalid Employee IDs: ['12477']
Row 7219: Invalid Employee IDs: ['1011804']
Row 341: Invalid Employee IDs: ['1020878']
Row 8176: Invalid Employee IDs: ['10277']
Row 8486: Invalid Employee IDs: ['20090']
Row 8487: Invalid Employee IDs: ['6']
Row 1314: Invalid Employee IDs: ['2878']
Row 9916: Invalid Employee IDs: ['20090']
Row 9982: Invalid Employee IDs: ['1011754']
Row 8486: Invalid Employee IDs: ['20090']
Row 6347: Inva

,Done By
0,None
1,None
2,None
3,None
4,None
...,...
12568,None
12569,None
12570,None
12571,None


In [ ]:
# Define a list of valid Employee IDs to use for replacement
valid_employee_ids = ['102477', '102878', '101325', '100140', '101754',
                      '102415', '103472', '102480', '103405', '200090',
                      '101664', '101916', '103471', '102817', '103422',
                      '100115', '103546', '102546', '102816', '101711',
                      '103481', '103581', '103580', '102871']

# Function to replace invalid Employee IDs with random valid IDs
def replace_invalid_employee_ids(employee_ids):
    ids = employee_ids.split(',')  # Split the Employee IDs by comma
    new_ids = []

    for eid in ids:
        cleaned_id = eid.strip()  # Remove whitespace
        if len(cleaned_id) != 6:  # Check if the ID is invalid
            # Replace with a random valid ID
            new_id = random.choice(valid_employee_ids)
            new_ids.append(new_id)
        else:
            new_ids.append(cleaned_id)  # Keep the valid ID

    return ', '.join(new_ids)  # Join the IDs back into a string

# Apply the function to the 'Employee ID' column
df['Done By'] = df['Done By'].apply(replace_invalid_employee_ids)

# Remove all white spaces from the 'Done By' column
df['Done By'] = df['Done By'].str.replace(r'\s+', '', regex=True)

df['Done By']

,Done By
0,102480
1,101754
2,102477
3,102477
4,101754
...,...
12568,200090
12569,200090
12570,101664
12571,102415


In [ ]:
# Function to check for invalid Employee IDs and print them with their row index
def print_invalid_employee_ids(employee_ids, index):
    ids = employee_ids.split(',')  # Split the Employee IDs by comma
    invalid_ids = [eid.strip() for eid in ids if len(eid.strip()) != 6]  # Collect IDs that are not 6 characters

    if invalid_ids:  # If there are invalid IDs, print them with the row index
        print(f"Row {index}: Invalid Employee IDs: {invalid_ids}")

# Apply the function to the 'Employee ID' column, passing the index as well
df['Done By'].apply(lambda x: print_invalid_employee_ids(x, df[df['Done By'] == x].index[0]))

,Done By
0,None
1,None
2,None
3,None
4,None
...,...
12568,None
12569,None
12570,None
12571,None


In [ ]:
df['Done By'] = df['Done By'].astype(str)
# df['Split ID'] = df['Split ID'].astype(str)
df['Lot ID'] = df['Lot ID'].astype(str)
df['Date Code'] = df['Date Code'].astype(str)
df['Date Code Input'] = df['Date Code Input'].astype(str)

In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Filter the DataFrame to show only rows with invalid Lot IDs
invalid_lot_ids = df[mask_invalid_lot_id]

# Display the invalid Lot IDs along with their corresponding rows
print(invalid_lot_ids[['Lot ID']])

              Lot ID
1277     CCN00312880
1458     CCN00312881
1497     12103345(B)
1526     CCN00312879
1594     12114264(B)
...              ...
12397  12566054(B/F)
12409  12609028(B/F)
12470  12554937(B/F)
12504  12586376(B/F)
12530  12609028(B/F)

[67 rows x 1 columns]


In [ ]:
# Create a mask for rows where 'Lot ID' contains '(' and is invalid (i.e., not an 8-digit number)
mask_invalid_lot_id_with_parenthesis = df['Lot ID'].astype(str).str.contains(r'\(|\)', regex=True)

# Filter the DataFrame to show only rows with invalid Lot IDs containing '('
invalid_lot_ids_with_parenthesis = df[mask_invalid_lot_id_with_parenthesis]

# Display the invalid Lot IDs along with their corresponding 'For 4Wire' data
print(invalid_lot_ids_with_parenthesis[['Lot ID', 'For 4Wire']])

              Lot ID  For 4Wire
1497     12103345(B)  BOARDFORM
1594     12114264(B)  BOARDFORM
1650     12114265(B)  BOARDFORM
1748     12067500(B)  BOARDFORM
1791     12140270(B)  BOARDFORM
...              ...        ...
12397  12566054(B/F)        NaN
12409  12609028(B/F)  BOARDFORM
12470  12554937(B/F)  BOARDFORM
12504  12586376(B/F)  BOARDFORM
12530  12609028(B/F)  BOARDFORM

[64 rows x 2 columns]


In [ ]:
# Create a mask for rows where 'Lot ID' contains '(' and is invalid (i.e., not an 8-digit number)
mask_invalid_lot_id_with_parenthesis = df['Lot ID'].astype(str).str.contains(r'\(|\)', regex=True)

# Filter the DataFrame to show only rows with invalid Lot IDs containing '('
invalid_lot_ids_with_parenthesis = df[mask_invalid_lot_id_with_parenthesis]

# Display the invalid Lot IDs along with their corresponding 'For 4Wire' data
print(invalid_lot_ids_with_parenthesis[['Lot ID', 'For 4Wire']])

              Lot ID  For 4Wire
1497     12103345(B)  BOARDFORM
1594     12114264(B)  BOARDFORM
1650     12114265(B)  BOARDFORM
1748     12067500(B)  BOARDFORM
1791     12140270(B)  BOARDFORM
...              ...        ...
12397  12566054(B/F)        NaN
12409  12609028(B/F)  BOARDFORM
12470  12554937(B/F)  BOARDFORM
12504  12586376(B/F)  BOARDFORM
12530  12609028(B/F)  BOARDFORM

[64 rows x 2 columns]


In [ ]:
# Use regex to remove the content inside parentheses (e.g., '(B)', '(B/F)', etc.)
df['Lot ID'] = df['Lot ID'].str.replace(r'\s*\(.*?\)', '', regex=True)

# Check the updated 'Lot ID' column for the rows with invalid Lot IDs
print(df.loc[invalid_lot_ids_with_parenthesis.index, ['Lot ID']])

         Lot ID
1497   12103345
1594   12114264
1650   12114265
1748   12067500
1791   12140270
...         ...
12397  12566054
12409  12609028
12470  12554937
12504  12586376
12530  12609028

[64 rows x 1 columns]


In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Create a mask for rows where 'Lot ID' contains '('
mask_lot_id_with_parenthesis = df['Lot ID'].astype(str).str.contains(r'\(')

# Combine the masks to find invalid Lot IDs that do NOT contain '('
mask_invalid_lot_id_no_parenthesis = mask_invalid_lot_id & ~mask_lot_id_with_parenthesis

# Filter the DataFrame to show only rows with invalid Lot IDs that do not contain '('
invalid_lot_ids_no_parenthesis = df[mask_invalid_lot_id_no_parenthesis]

# Display the invalid Lot IDs without '(' along with their corresponding rows
print(invalid_lot_ids_no_parenthesis[['Lot ID', 'For 4Wire']])

           Lot ID For 4Wire
1277  CCN00312880       NaN
1458  CCN00312881       NaN
1526  CCN00312879       NaN


In [ ]:
# Create a mask for rows where 'Lot ID' starts with 'CCN'
mask_ccn_lot_ids = df['Lot ID'].str.startswith('CCN')

# Apply the split operation only to rows where the 'Lot ID' starts with 'CCN'
df.loc[mask_ccn_lot_ids, 'Split ID'] = df.loc[mask_ccn_lot_ids, 'Lot ID'].str[:3]    # Extract 'CCN' into 'Split ID'
df.loc[mask_ccn_lot_ids, 'Lot ID'] = df.loc[mask_ccn_lot_ids, 'Lot ID'].str[3:]      # Keep the rest in 'Lot ID'

# Ensure 'Split ID' remains with 'NaN' values for missing entries, not 'nan'
df['Split ID'] = df['Split ID'].fillna(np.NaN)

# Verify the changes by printing the updated DataFrame
print(df.loc[1277])

Added Time                     2023-08-19 00:00:00
Done By                                     100140
Lot No.                                CCN00312880
Tool Number (S. No.)                      S11795_A
Kevin Probe Test                                NO
For 4Wire                                      NaN
DateCode                                      2228
Job Type                                       RMA
Machine                                      KAIMA
X-Cut Allowed                                  0.0
IPC Class                                      3.0
Mass Lam                                        NO
Resistive                                       NO
Continuity                                      10
Test Voltage                                   250
Isolation                                       10
Adjacency Used ( Emma Only)                    YES
Mfg Qty                                         71
Tested Qty                                      71
1st Passed Qty                 

In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Create a mask for rows where 'Lot ID' contains '('
mask_lot_id_with_parenthesis = df['Lot ID'].astype(str).str.contains(r'\(')

# Combine the masks to find invalid Lot IDs that do NOT contain '('
mask_invalid_lot_id_no_parenthesis = mask_invalid_lot_id & ~mask_lot_id_with_parenthesis

# Filter the DataFrame to show only rows with invalid Lot IDs that do not contain '('
invalid_lot_ids_no_parenthesis = df[mask_invalid_lot_id_no_parenthesis]

# Display the invalid Lot IDs without '(' along with their corresponding rows
print(invalid_lot_ids_no_parenthesis[['Lot ID', 'For 4Wire']])

Empty DataFrame
Columns: [Lot ID, For 4Wire]
Index: []


In [ ]:
# Create a mask for rows where 'Lot ID' contains '(' or ')' and does not have exactly 8 characters
mask_invalid_lot_id = df['Lot ID'].astype(str).str.contains(r'\(|\)', regex=True) & (df['Lot ID'].astype(str).str.len() != 8)

# Filter the DataFrame to show only rows that meet both conditions
invalid_lot_ids = df[mask_invalid_lot_id]

# Display the invalid Lot IDs along with their corresponding 'For 4Wire' data
print(invalid_lot_ids[['Lot ID']])

Empty DataFrame
Columns: [Lot ID]
Index: []


In [ ]:
df.head()

,Added Time,Done By,Lot No.,Tool Number (S. No.),Kevin Probe Test,For 4Wire,DateCode,Job Type,Machine,X-Cut Allowed,...,Open,Short,SDP,Lot ID Suffix,Full Lot ID,Standardized Date,Split ID,Lot ID,Date Code,Date Code Input
0,2023-01-01 01:30:09,102480,11387972,S03804_A,NO,NaN,2252,STANDARD,VERSA,3.0,...,63,64,38,NaN,11387972,2023-01-01,NaN,11387972,Yes,2252
1,2023-07-01 00:38:13,101754,12105804,S11664_E,NO,NaN,2623,QTA,KAIMA,0.0,...,0,0,29,NaN,12105804,2023-07-01,NaN,12105804,Yes,2623
2,2023-07-01 03:19:06,102477,12107143,S12881_B,NO,NaN,2623,NPI,6151 (1),1.0,...,0,0,6,NaN,12107143,2023-07-01,NaN,12107143,Yes,2623
3,2023-07-01 03:21:07,102477,12107226,S12949_A,NO,NaN,2523,QTA,4033,0.0,...,4,0,10,NaN,12107226,2023-07-01,NaN,12107226,Yes,2523
4,2023-07-01 05:13:25,101754,12073207,S11515_J,NO,NaN,2623,STANDARD,KAIMA,0.0,...,40,0,56,NaN,12073207,2023-07-01,NaN,12073207,Yes,2623


In [ ]:
# Function to concatenate columns while ignoring NaN values
def combine_lot_ids(row):
    parts = [row['Split ID'], row['Lot ID'], row['Lot ID Suffix']]
    # Filter out None/NaN values and join the remaining parts with a separator
    return ''.join(str(part) for part in parts if pd.notna(part))

# Apply the function to create the 'Full Lot ID'
df['Full Lot ID'] = df.apply(combine_lot_ids, axis=1)

# Display the DataFrame with the new 'Full Lot ID'
print(df[df['Lot ID'] == '12222598'])

     Added Time               Done By   Lot No. Tool Number (S. No.)  \
2949 2023-11-01  102477,103472,102878  12222598             S12683_B   
2999 2023-11-04                101754  12222598             S12683_B   
3087 2023-11-06                101754  12222598             S12683_B   

     Kevin Probe Test  For 4Wire                DateCode      Job Type  \
2949              YES  PANELFORM  NO DATE CODE AVAILABLE  IST REQUIRED   
2999               NO        NaN                    4423  IST REQUIRED   
3087               NO        NaN                    4423  IST REQUIRED   

       Machine X-Cut Allowed  ... Open Short SDP  Lot ID Suffix  \
2949  6151 (3)           0.0  ...    1     0  20        (4WIRE)   
2999     KAIMA           0.0  ...   86     0  21            NaN   
3087     KAIMA           0.0  ...   15    43  21            NaN   

          Full Lot ID  Standardized Date Split ID    Lot ID  Date Code  \
2949  12222598(4WIRE)         2023-11-01      NaN  12222598         No 

In [ ]:
# Check duplicates in your main dataframe
duplicated_lot_nos = df[df.duplicated(subset='Full Lot ID', keep=False)]['Full Lot ID'].unique()
print(duplicated_lot_nos)
print("Total number of duplicated 'Lot No.':", len(duplicated_lot_nos))

['12019282' '12078158' '12115138' '12092076' '11997106(680)'
 '11997104(680)' '12130396(680, TEST 2)' '12103345(4WIRE)'
 '12067484(4WIRE)' '12067499(4WIRE)' '12080128' '12067500(4WIRE)'
 '12104050' '12127759' '12078203' '12140270(4WIRE)' '12170333(4WIRE)'
 '12150981' '12150830(4WIRE)' '12113172(4WIRE)' '12172442'
 '12119211(4WIRE)' '12160620' '12103424(4WIRE)' '12098221(4WIRE)'
 '12118896' '12131997(4WIRE)' '12140284(4WIRE)' '12144295(4WIRE)'
 '12144241(4WIRE)' '12144239(4WIRE)' '12144240(4WIRE)' '12173994(SILVER)'
 '12193099(4WIRE)' '12191975' '12110334' '12191386' '12133579' '12222598'
 '12052039' '12223469' '12221363' '12195873' '12195748' '12211821'
 '12212456' '12229178' '12212460' '12255338' '12272801' '12272610'
 '12253539' '12246262' '12272665' '12287651' '12266665' '12297165'
 '12253619(SILVER)' '12294598' '12269460' '12272851' '12318673' '12300324'
 '12291181' '12314696' '12314794' '12314700' '12333448' '12354197'
 '12358728' '12373089(4WIRE)' '12373113(4WIRE)' '12343109'
 '1

In [ ]:
# Specify columns to ignore when checking for duplicates
cols_to_ignore = ['Standardized Date', 'Done By']

# Create a list of columns to consider for dropping duplicates (all except the ignored ones)
cols_to_check = df.columns.difference(cols_to_ignore)

# Drop duplicates based on the specified columns
df_cleaned = df.drop_duplicates(subset=cols_to_check, keep='last')

# Optional: Reset index if needed
df_cleaned.reset_index(drop=True, inplace=True)

# Print the cleaned DataFrame
df_cleaned

,Added Time,Done By,Lot No.,Tool Number (S. No.),Kevin Probe Test,For 4Wire,DateCode,Job Type,Machine,X-Cut Allowed,...,Open,Short,SDP,Lot ID Suffix,Full Lot ID,Standardized Date,Split ID,Lot ID,Date Code,Date Code Input
0,2023-01-01 01:30:09,102480,11387972,S03804_A,NO,NaN,2252,STANDARD,VERSA,3.0,...,63,64,38,NaN,11387972,2023-01-01,NaN,11387972,Yes,2252
1,2023-07-01 00:38:13,101754,12105804,S11664_E,NO,NaN,2623,QTA,KAIMA,0.0,...,0,0,29,NaN,12105804,2023-07-01,NaN,12105804,Yes,2623
2,2023-07-01 03:19:06,102477,12107143,S12881_B,NO,NaN,2623,NPI,6151 (1),1.0,...,0,0,6,NaN,12107143,2023-07-01,NaN,12107143,Yes,2623
3,2023-07-01 03:21:07,102477,12107226,S12949_A,NO,NaN,2523,QTA,4033,0.0,...,4,0,10,NaN,12107226,2023-07-01,NaN,12107226,Yes,2523
4,2023-07-01 05:13:25,101754,12073207,S11515_J,NO,NaN,2623,STANDARD,KAIMA,0.0,...,40,0,56,NaN,12073207,2023-07-01,NaN,12073207,Yes,2623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12560,2025-01-08 13:03:24,200090,12587705,S13203_A,NO,NaN,0225,STANDARD,KAIMA,1.0,...,1,1,5,NaN,12587705,2025-01-08,NaN,12587705,Yes,0225
12561,2025-01-08 14:45:20,200090,12587392,S12453_A,NO,NaN,2452,STANDARD,KAIMA,0.0,...,1,2,27,(SILVER),12587392(SILVER),2025-01-08,NaN,12587392,Yes,2452
12562,2025-01-08 16:12:20,101664,12588590,S12695_B,NO,NaN,0225,QTA,6151 (2),0.0,...,1,0,31,(SILVER),12588590(SILVER),2025-01-08,NaN,12588590,Yes,0225
12563,2025-01-08 16:14:55,102415,12591568,S09018_C,NO,NaN,0225,STANDARD,LM1,0.0,...,1,0,5,NaN,12591568,2025-01-08,NaN,12591568,Yes,0225


In [ ]:
# Check duplicates in your main dataframe
duplicated_lot_nos = df_cleaned[df_cleaned.duplicated(subset='Full Lot ID', keep=False)]['Full Lot ID'].unique()
print(duplicated_lot_nos)
print("Total number of duplicated 'Lot No.':", len(duplicated_lot_nos))

['12019282' '12078158' '12115138' '12092076' '11997106(680)'
 '11997104(680)' '12130396(680, TEST 2)' '12103345(4WIRE)'
 '12067484(4WIRE)' '12067499(4WIRE)' '12080128' '12067500(4WIRE)'
 '12104050' '12140270(4WIRE)' '12170333(4WIRE)' '12150981'
 '12150830(4WIRE)' '12113172(4WIRE)' '12119211(4WIRE)' '12160620'
 '12103424(4WIRE)' '12098221(4WIRE)' '12118896' '12131997(4WIRE)'
 '12140284(4WIRE)' '12144295(4WIRE)' '12144241(4WIRE)' '12144239(4WIRE)'
 '12144240(4WIRE)' '12173994(SILVER)' '12193099(4WIRE)' '12191975'
 '12110334' '12191386' '12133579' '12222598' '12052039' '12221363'
 '12195873' '12211821' '12212456' '12229178' '12212460' '12255338'
 '12272801' '12272610' '12253539' '12246262' '12272665' '12287651'
 '12266665' '12297165' '12253619(SILVER)' '12294598' '12269460' '12318673'
 '12300324' '12291181' '12314696' '12314794' '12314700' '12354197'
 '12358728' '12373089(4WIRE)' '12373113(4WIRE)' '12343109'
 '12355347(4WIRE)' '12380069(4WIRE)' '12385520' '12372999(4WIRE)'
 '12411881' '12

In [ ]:
# Sort by 'Added Time' in ascending order to keep the latest entry
df_sorted = df_cleaned.sort_values(by='Added Time', ascending=False)

# Drop duplicates based on 'Full Lot No', keeping only the latest occurrence
df_cleaned = df_sorted.drop_duplicates(subset='Full Lot ID')

# Sort the DataFrame again based on 'Added Time' in ascending order
df_cleaned = df_cleaned.sort_values(by='Added Time', ascending=True)

# Optional: Reset index if needed
df_cleaned.reset_index(drop=True, inplace=True)

# Check the cleaned DataFrame
df_cleaned.head()

,Added Time,Done By,Lot No.,Tool Number (S. No.),Kevin Probe Test,For 4Wire,DateCode,Job Type,Machine,X-Cut Allowed,...,Open,Short,SDP,Lot ID Suffix,Full Lot ID,Standardized Date,Split ID,Lot ID,Date Code,Date Code Input
0,2023-01-01 01:30:09,102480,11387972,S03804_A,NO,NaN,2252,STANDARD,VERSA,3.0,...,63,64,38,NaN,11387972,2023-01-01,NaN,11387972,Yes,2252
1,2023-07-01 00:38:13,101754,12105804,S11664_E,NO,NaN,2623,QTA,KAIMA,0.0,...,0,0,29,NaN,12105804,2023-07-01,NaN,12105804,Yes,2623
2,2023-07-01 03:19:06,102477,12107143,S12881_B,NO,NaN,2623,NPI,6151 (1),1.0,...,0,0,6,NaN,12107143,2023-07-01,NaN,12107143,Yes,2623
3,2023-07-01 03:21:07,102477,12107226,S12949_A,NO,NaN,2523,QTA,4033,0.0,...,4,0,10,NaN,12107226,2023-07-01,NaN,12107226,Yes,2523
4,2023-07-01 05:13:25,101754,12073207,S11515_J,NO,NaN,2623,STANDARD,KAIMA,0.0,...,40,0,56,NaN,12073207,2023-07-01,NaN,12073207,Yes,2623


In [ ]:
# Check duplicates in your main dataframe
duplicated_lot_nos =df_cleaned[df_cleaned.duplicated(subset='Full Lot ID', keep=False)]['Full Lot ID'].unique()
print(duplicated_lot_nos)
print("Total number of duplicated 'Lot No.':", len(duplicated_lot_nos))

[]
Total number of duplicated 'Lot No.': 0


In [ ]:
# Print the number of rows before and after dropping duplicates
print(f"Original DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape: {df_cleaned.shape}")

Original DataFrame shape: (12573, 30)
Cleaned DataFrame shape: (12409, 30)


In [ ]:
selected_cols = [
    'Standardized Date',
    'Done By',
    'Split ID',
    'Lot ID',
    'Tool Number (S. No.)',
    'Kevin Probe Test',
    'For 4Wire',
    'Date Code',
    'Date Code Input',
    'Job Type',
    'Machine',
    'X-Cut Allowed',
    'IPC Class',
    'Mass Lam',
    'Resistive',
    'Continuity',
    'Test Voltage',
    'Isolation',
    'Adjacency Used ( Emma Only)',
    'Lot ID Suffix',
    'Mfg Qty',
    'Tested Qty',
    '1st Passed Qty',
    'Open',
    'Short',
    'SDP',
    'Full Lot ID'
]

new_df = df_cleaned[selected_cols]

new_df.head()

,Standardized Date,Done By,Split ID,Lot ID,Tool Number (S. No.),Kevin Probe Test,For 4Wire,Date Code,Date Code Input,Job Type,...,Isolation,Adjacency Used ( Emma Only),Lot ID Suffix,Mfg Qty,Tested Qty,1st Passed Qty,Open,Short,SDP,Full Lot ID
0,2023-01-01,102480,NaN,11387972,S03804_A,NO,NaN,Yes,2252,STANDARD,...,10,NO,NaN,630,592,465,63,64,38,11387972
1,2023-07-01,101754,NaN,12105804,S11664_E,NO,NaN,Yes,2623,QTA,...,10,NO,NaN,120,91,91,0,0,29,12105804
2,2023-07-01,102477,NaN,12107143,S12881_B,NO,NaN,Yes,2623,NPI,...,1048,YES,NaN,48,42,42,0,0,6,12107143
3,2023-07-01,102477,NaN,12107226,S12949_A,NO,NaN,Yes,2523,QTA,...,10,YES,NaN,336,326,322,4,0,10,12107226
4,2023-07-01,101754,NaN,12073207,S11515_J,NO,NaN,Yes,2623,STANDARD,...,10,NO,NaN,432,376,336,40,0,56,12073207


In [ ]:
new_df['Done By'] = new_df['Done By'].astype(str)

<ipython-input-50-da6841278ba5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Done By'] = new_df['Done By'].astype(str)


In [ ]:
new_df.dtypes

,0
Standardized Date,datetime64[ns]
Done By,object
Split ID,object
Lot ID,object
Tool Number (S. No.),object
Kevin Probe Test,object
For 4Wire,object
Date Code,object
Date Code Input,object
Job Type,object


# **df1**

In [ ]:
df1 = get_as_dataframe(sheet1, evaluate_formulas=True, headers=True)

# Drop the headers and reset the index
df1 = df1.iloc[1:].reset_index(drop=True)

# Rename columns with custom header names
custom_headers = ['Added Time', 'IP Address', 'Employee ID', 'Lot ID', '1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair', 'Short - Qty Reject Photoprint 083 OL_PP_DEV', '2. Scatches Qty Repair', 'Scratches Qty Reject - Photoprint 483_OL_PP_DEV',
                  'Scratches Qty Reject CHEM B 483 OL_CM_STRP', 'Scratches Qty Reject CHEM C 483 OL_CM_OSP', '3. Cu Residue Qty Repair', 'Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP', '4. Under-etch Qty Repair', 'Under-etch Qty Reject CHEM B 076 OL_CM_STRP',
                  '5. Innerlayer Short - Qty Reject 083 IL_STRIP', '6. Mis-registration Qty Repair', 'Mis-Registration - Qty Reject 471 OL_LAM', 'Mis-Registration - Qty Reject 469 OL_LAM', '7. Hole Shifted Qty Reject 132 OL_DRL', '8. Short (Micro-short) Qty Repair',
                  'Short (Micro-short) Qty Reject CHEM B 083 OL_CM_STRP', 'Short (Micro-short) Qty Reject LPSM 083 OL_SM_CURE', '9. Incomplete Solder Strip (Solder Short) Qty Repair', 'Incomplete Solder Strip (Solder Short) Qty Reject CHEM C 256 OL_CM_OSP',
                  '10. Feathering (AU Bridging) Qty Repair', 'Feathering (AU Bridging) Qty Reject CHEM C 250 OL_CM_OSP', '11. Incomplete Resist Strip -Qty Repair', 'Incomplete Resist Strip Qty Reject CHEM B OL_CM_STRP', '12. NPTH Short Qty Repair', 'NPTH Short Qty Reject 142',
                  '13. Short (Others) Qty Repair', 'Short (Others) Qty Reject', '14. Pit & Dent (Others) Qty Reject 479 OL_LAM', 'Pit & Dent (Others) Qty Reject 479 Ol_PP_DEV', '15. Foil Wrinkle ( Others ) Qty Reject 32 OL_LAM', '1. Innerlayer Open Qty Reject 080 IL_STRIP',
                  '2. Void Hole Qty Reject CHEM A 180 OL_CM_DM_O', '3. Partial Feature (Blind Via Void) Qty Reject 477 OL_CM_DM_O', '4. Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_LPSM_CURE', 'Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_CM_OSP',
                  'Lifted Feature (Circuit Peel Off) Qty Reject 473  OL_CM_STRP', '5. Missing Hole Qty Reject 133 OL_DRL', '6. Outerlayer Open Qty Repair', 'Outerlayer Open Qty Reject', '7. Mask on Pad / Finger Qty Repair', 'Mask On Pad/Finger Qty Reject 205 OL_SM_CURE',
                  '8. Plating Nodule (Particle in Hole)', 'Plating Nodule (Particle In Hole) Qty Reject 178 OL_CM_STRP', '9. Legend On Pad /Hole Qty Repair', 'Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE', '10. Others Open Qty Repair', 'Others Open Qty Reject',
                  '11. False o/c Qty Repair', 'Impedance Fail (Y/N)', 'High', 'Low', 'Final Passed Qty', 'Lot ID Suffix', 'a', 'b', 'c', '1st pass qty for BE test', 'tested qty for BE Test', 'Total After Final Short', 'Total After Final Open', 'TOTAL OL OPEN DEFECTS',
                  'TOTAL OL SHORT DEFECTS', 'TOTAL IL SHORT DEFECTS', 'TOTAL IL OPEN DEFECTS', 'TOTAL MISREGISTRATION', 'SCRATCH', 'VOID HOLE', 'OL SHORT REPAIR', 'MISREG REPAIR', 'SCRATCH REPAIR', 'Full Lot ID', 'Copy of Final QTY', 'Copy of TT short', 'Copy of TT Open']
df1.columns = custom_headers

df1 = df1.dropna(subset=['Added Time'])
df1

,Added Time,IP Address,Employee ID,Lot ID,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,Scratches Qty Reject CHEM B 483 OL_CM_STRP,Scratches Qty Reject CHEM C 483 OL_CM_OSP,...,TOTAL MISREGISTRATION,SCRATCH,VOID HOLE,OL SHORT REPAIR,MISREG REPAIR,SCRATCH REPAIR,Full Lot ID,Copy of Final QTY,Copy of TT short,Copy of TT Open
0,01-Jan-2022 1:26:53,165.225.230.207,102817.0,11487783,NaN,NaN,NaN,NaN,NaN,NaN,...,#REF! (Array result was not expanded because i...,0.0,0.0,0.0,0.0,0.0,11487783,3800,0,20
1,01-Jul-2023 3:17:08,165.225.116.126,102817.0,12105804,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12105804,91,0,0
2,01-Jul-2023 3:17:36,165.225.116.126,102817.0,12107143,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12107143,42,0,0
3,01-Jul-2023 3:19:43,165.225.116.126,102817.0,12107226,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12107226,322,0,4
4,01-Jul-2023 3:21:45,165.225.116.126,102817.0,12111643,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12111643,84,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12426,08-Jan-2025 13:10:12,165.225.230.186,100140.0,12593102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12593102,1108,0,3
12427,08-Jan-2025 15:30:28,165.225.230.186,100140.0,12587392,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12587392(Silver),510,0,3
12428,08-Jan-2025 16:01:33,165.225.230.186,100140.0,12587705,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12587705,2753,0,2
12429,08-Jan-2025 16:12:59,165.225.230.186,100140.0,12588590,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,12588590(Silver),88,0,1


In [ ]:
cols_to_drop = ['IP Address', 'a', 'b', 'c', '1st pass qty for BE test', 'tested qty for BE Test', 'Total After Final Short', 'Total After Final Open', 'TOTAL OL OPEN DEFECTS', 'TOTAL OL SHORT DEFECTS', 'TOTAL IL SHORT DEFECTS', 'TOTAL IL OPEN DEFECTS', 'TOTAL MISREGISTRATION', 'SCRATCH', 'VOID HOLE', 'OL SHORT REPAIR', 'MISREG REPAIR', 'SCRATCH REPAIR', 'Copy of Final QTY', 'Copy of TT short', 'Copy of TT Open']

# Drop the specified columns
df1 = df1.drop(columns=cols_to_drop)

df1.head()

,Added Time,Employee ID,Lot ID,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,Scratches Qty Reject CHEM B 483 OL_CM_STRP,Scratches Qty Reject CHEM C 483 OL_CM_OSP,3. Cu Residue Qty Repair,...,Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE,10. Others Open Qty Repair,Others Open Qty Reject,11. False o/c Qty Repair,Impedance Fail (Y/N),High,Low,Final Passed Qty,Lot ID Suffix,Full Lot ID
0,01-Jan-2022 1:26:53,102817.0,11487783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,3800.0,NaN,11487783
1,01-Jul-2023 3:17:08,102817.0,12105804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,91.0,NaN,12105804
2,01-Jul-2023 3:17:36,102817.0,12107143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,42.0,NaN,12107143
3,01-Jul-2023 3:19:43,102817.0,12107226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,322.0,NaN,12107226
4,01-Jul-2023 3:21:45,102817.0,12111643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,84.0,NaN,12111643


In [ ]:
# Convert all string columns to uppercase
df1 = df1.applymap(lambda x: x.upper() if isinstance(x, str) else x)

<ipython-input-54-346a9e1d8f66>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [ ]:
# Check for missing values in the DataFrame
missing_values = df1.isnull().sum()

# Display missing values for each column
if missing_values.any():
    print("Missing Values in Each Column:")
    print(missing_values[missing_values > 0])
else:
    print("There are no missing values in the dataset.")

Missing Values in Each Column:
1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair            12429
Short - Qty Reject Photoprint 083 OL_PP_DEV                               11476
2. Scatches Qty Repair                                                    12431
Scratches Qty Reject - Photoprint 483_OL_PP_DEV                           12429
Scratches Qty Reject CHEM B 483 OL_CM_STRP                                12427
Scratches Qty Reject CHEM C 483 OL_CM_OSP                                 12431
3. Cu Residue Qty Repair                                                  12431
Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP                             12032
4. Under-etch Qty Repair                                                  12431
Under-etch Qty Reject CHEM B 076 OL_CM_STRP                               11819
5. Innerlayer Short - Qty Reject 083 IL_STRIP                              7628
6. Mis-registration Qty Repair                                            12428
Mis-Regis

In [ ]:
df1['Added Time'] = df1['Added Time'].astype(str)
df1['Employee ID'] = df1['Employee ID'].astype(str)
df1['Lot ID'] = df1['Lot ID'].astype(str)
df1['Added Time'] = pd.to_datetime(df1['Added Time'], format='%d-%b-%Y %H:%M:%S', errors='coerce')

In [ ]:
# Ensure 'Standardized Date' is a copy of 'Added Time' in datetime format (no formatting yet)
df1['Standardized Date'] = df1['Added Time']

# Convert 'Standardized Date' to remove the time component but keep as datetime64
df1['Standardized Date'] = pd.to_datetime(df1['Standardized Date']).dt.normalize()

# Check for NaT values
num_nat = df1['Added Time'].isna().sum()
print(f"Number of NaT values after custom parsing: {num_nat}")

# Display the DataFrame
print(df1[['Added Time', 'Standardized Date']].head(20))

Number of NaT values after custom parsing: 0
            Added Time Standardized Date
0  2022-01-01 01:26:53        2022-01-01
1  2023-07-01 03:17:08        2023-07-01
2  2023-07-01 03:17:36        2023-07-01
3  2023-07-01 03:19:43        2023-07-01
4  2023-07-01 03:21:45        2023-07-01
5  2023-07-01 03:23:41        2023-07-01
6  2023-07-01 06:12:02        2023-07-01
7  2023-07-01 06:13:48        2023-07-01
8  2023-07-01 08:29:08        2023-07-01
9  2023-07-01 09:21:40        2023-07-01
10 2023-07-01 10:07:12        2023-07-01
11 2023-07-01 10:14:16        2023-07-01
12 2023-07-01 10:50:38        2023-07-01
13 2023-07-01 16:21:08        2023-07-01
14 2023-07-01 16:40:49        2023-07-01
15 2023-07-01 16:45:02        2023-07-01
16 2023-07-01 18:05:47        2023-07-01
17 2023-07-01 18:10:09        2023-07-01
18 2023-07-01 18:28:52        2023-07-01
19 2023-07-02 03:25:16        2023-07-02


In [ ]:
nat_count = df1['Added Time'].isna().sum()
print(f'Number of NaT values in "Added Time": {nat_count}')

# Optionally, check for NaT in the 'Standardized Date' column
standardized_nat_count = df1['Standardized Date'].isna().sum()
print(f'Number of NaT values in "Standardized Date": {standardized_nat_count}')

Number of NaT values in "Added Time": 0
Number of NaT values in "Standardized Date": 0


In [ ]:
nat_rows = df1[df1['Added Time'].isna() | df1['Standardized Date'].isna()]

# Display these rows to check the original values
print(nat_rows)

Empty DataFrame
Columns: [Added Time, Employee ID, Lot ID, 1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair, Short - Qty Reject Photoprint 083 OL_PP_DEV, 2. Scatches Qty Repair, Scratches Qty Reject - Photoprint 483_OL_PP_DEV, Scratches Qty Reject CHEM B 483 OL_CM_STRP, Scratches Qty Reject CHEM C 483 OL_CM_OSP, 3. Cu Residue Qty Repair, Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP, 4. Under-etch Qty Repair, Under-etch Qty Reject CHEM B 076 OL_CM_STRP, 5. Innerlayer Short - Qty Reject 083 IL_STRIP, 6. Mis-registration Qty Repair, Mis-Registration - Qty Reject 471 OL_LAM, Mis-Registration - Qty Reject 469 OL_LAM, 7. Hole Shifted Qty Reject 132 OL_DRL, 8. Short (Micro-short) Qty Repair, Short (Micro-short) Qty Reject CHEM B 083 OL_CM_STRP, Short (Micro-short) Qty Reject LPSM 083 OL_SM_CURE, 9. Incomplete Solder Strip (Solder Short) Qty Repair, Incomplete Solder Strip (Solder Short) Qty Reject CHEM C 256 OL_CM_OSP, 10. Feathering (AU Bridging) Qty Repair, Feathering (AU Bri

In [ ]:
# Replace the NaT value in 'Added Time' column at index 7534
df1.at[7534 , 'Added Time'] = '2024-05-27 9:53:11'

# Ensure 'Standardized Date' remains a normalized datetime (without time component)
df1['Standardized Date'] = pd.to_datetime(df1['Added Time']).dt.normalize()

# Check for NaT values after the update
num_nat = df1['Added Time'].isna().sum()
print(f"Number of NaT values after custom parsing: {num_nat}")

# Display the DataFrame for verification
print(df1.loc[7530:7540, ['Added Time', 'Standardized Date']])

Number of NaT values after custom parsing: 0
              Added Time Standardized Date
7530 2024-05-22 06:20:19        2024-05-22
7531 2024-05-22 06:22:24        2024-05-22
7532 2024-05-22 06:22:45        2024-05-22
7533 2024-05-22 06:23:08        2024-05-22
7534 2024-05-27 09:53:11        2024-05-27
7535 2024-05-27 10:12:40        2024-05-27
7536 2024-05-27 10:26:42        2024-05-27
7537 2024-05-27 11:21:52        2024-05-27
7538 2024-05-27 14:43:41        2024-05-27
7539 2024-05-27 15:15:17        2024-05-27
7540 2024-05-27 15:49:33        2024-05-27


In [ ]:
nat_count = df1['Added Time'].isna().sum()
print(f'Number of NaT values in "Added Time": {nat_count}')

# Optionally, check for NaT in the 'Standardized Date' column
standardized_nat_count = df1['Standardized Date'].isna().sum()
print(f'Number of NaT values in "Standardized Date": {standardized_nat_count}')

Number of NaT values in "Added Time": 0
Number of NaT values in "Standardized Date": 0


In [ ]:
columns_to_fill = [
    "1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair",
    "Short - Qty Reject Photoprint 083 OL_PP_DEV",
    "2. Scatches Qty Repair",
    "Scratches Qty Reject - Photoprint 483_OL_PP_DEV",
    "Scratches Qty Reject CHEM B 483 OL_CM_STRP",
    "Scratches Qty Reject CHEM C 483 OL_CM_OSP",
    "3. Cu Residue Qty Repair",
    "Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP",
    "4. Under-etch Qty Repair",
    "Under-etch Qty Reject CHEM B 076 OL_CM_STRP",
    "5. Innerlayer Short - Qty Reject 083 IL_STRIP",
    "6. Mis-registration Qty Repair",
    "Mis-Registration - Qty Reject 471 OL_LAM",
    "Mis-Registration - Qty Reject 469 OL_LAM",
    "7. Hole Shifted Qty Reject 132 OL_DRL",
    "8. Short (Micro-short) Qty Repair",
    "Short (Micro-short) Qty Reject CHEM B 083 OL_CM_STRP",
    "Short (Micro-short) Qty Reject LPSM 083 OL_SM_CURE",
    "9. Incomplete Solder Strip (Solder Short) Qty Repair",
    "Incomplete Solder Strip (Solder Short) Qty Reject CHEM C 256 OL_CM_OSP",
    "10. Feathering (AU Bridging) Qty Repair",
    "Feathering (AU Bridging) Qty Reject CHEM C 250 OL_CM_OSP",
    "11. Incomplete Resist Strip -Qty Repair",
    "Incomplete Resist Strip Qty Reject CHEM B OL_CM_STRP",
    "12. NPTH Short Qty Repair",
    "NPTH Short Qty Reject 142",
    "13. Short (Others) Qty Repair",
    "Short (Others) Qty Reject",
    "14. Pit & Dent (Others) Qty Reject 479 OL_LAM",
    "Pit & Dent (Others) Qty Reject 479 Ol_PP_DEV",
    "15. Foil Wrinkle ( Others ) Qty Reject 32 OL_LAM",
    "1. Innerlayer Open Qty Reject 080 IL_STRIP",
    "2. Void Hole Qty Reject CHEM A 180 OL_CM_DM_O",
    "3. Partial Feature (Blind Via Void) Qty Reject 477 OL_CM_DM_O",
    "4. Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_LPSM_CURE",
    "Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_CM_OSP",
    "Lifted Feature (Circuit Peel Off) Qty Reject 473  OL_CM_STRP",
    "5. Missing Hole Qty Reject 133 OL_DRL",
    "6. Outerlayer Open Qty Repair",
    "Outerlayer Open Qty Reject",
    "7. Mask on Pad / Finger Qty Repair",
    "Mask On Pad/Finger Qty Reject 205 OL_SM_CURE",
    "8. Plating Nodule (Particle in Hole)",
    "Plating Nodule (Particle In Hole) Qty Reject 178 OL_CM_STRP",
    "9. Legend On Pad /Hole Qty Repair",
    "Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE",
    "10. Others Open Qty Repair",
    "Others Open Qty Reject",
    "11. False o/c Qty Repair",
    "Final Passed Qty"
]

# Replace NaN with 0 in the specified columns
df1[columns_to_fill] = df1[columns_to_fill].fillna(0).astype(int)

# Display the DataFrame to verify the changes
df1.head()

,Added Time,Employee ID,Lot ID,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,Scratches Qty Reject CHEM B 483 OL_CM_STRP,Scratches Qty Reject CHEM C 483 OL_CM_OSP,3. Cu Residue Qty Repair,...,10. Others Open Qty Repair,Others Open Qty Reject,11. False o/c Qty Repair,Impedance Fail (Y/N),High,Low,Final Passed Qty,Lot ID Suffix,Full Lot ID,Standardized Date
0,2022-01-01 01:26:53,102817.0,11487783,0,0,0,0,0,0,0,...,0,0,0,NO,NaN,NaN,3800,NaN,11487783,2022-01-01
1,2023-07-01 03:17:08,102817.0,12105804,0,0,0,0,0,0,0,...,0,0,0,NO,NaN,NaN,91,NaN,12105804,2023-07-01
2,2023-07-01 03:17:36,102817.0,12107143,0,0,0,0,0,0,0,...,0,0,0,NO,NaN,NaN,42,NaN,12107143,2023-07-01
3,2023-07-01 03:19:43,102817.0,12107226,0,0,0,0,0,0,0,...,0,0,0,NO,NaN,NaN,322,NaN,12107226,2023-07-01
4,2023-07-01 03:21:45,102817.0,12111643,0,0,0,0,0,0,0,...,0,0,0,NO,NaN,NaN,84,NaN,12111643,2023-07-01


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12431 entries, 0 to 12430
Data columns (total 59 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Added Time                                                              12431 non-null  datetime64[ns]
 1   Employee ID                                                             12431 non-null  object        
 2   Lot ID                                                                  12431 non-null  object        
 3   1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair          12431 non-null  int64         
 4   Short - Qty Reject Photoprint 083 OL_PP_DEV                             12431 non-null  int64         
 5   2. Scatches Qty Repair                                                  12431 non-null  int64         
 6   Scratches Qty Reject -

In [ ]:
# Define a function to split the 'Lot No.' into 'Split ID' and 'Lot ID'
def split_lot_no(lot_no):
    parts = lot_no.split(', ')
    if len(parts) == 2:
        return parts[0], parts[1]  # Return Split ID and Lot ID
    else:
        return np.NaN, parts[0]  # No Split ID, return None and the whole Lot ID

# Apply the function to the 'Lot No.' column
df1[['Split ID', 'Lot ID']] = df1['Lot ID'].apply(split_lot_no).apply(pd.Series)

df1.loc[7538]

,7538
Added Time,2024-05-27 14:43:41
Employee ID,100416.0
Lot ID,12407648
1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,0
Short - Qty Reject Photoprint 083 OL_PP_DEV,0
2. Scatches Qty Repair,0
Scratches Qty Reject - Photoprint 483_OL_PP_DEV,0
Scratches Qty Reject CHEM B 483 OL_CM_STRP,0
Scratches Qty Reject CHEM C 483 OL_CM_OSP,0
3. Cu Residue Qty Repair,0


In [ ]:
# Remove the ".0" suffix if present and ensure 'Done By' is a string
df1['Employee ID'] = df1['Employee ID'].astype(str).str.replace(r'\.0$', '', regex=True)

# Replace '/' with ',' in the specified column
df1['Employee ID'] = df1['Employee ID'].str.replace(r'\s*/\s*', ',', regex=True)

In [ ]:
# Function to check for invalid Employee IDs and print them with their row index
def print_invalid_employee_ids(employee_ids, index):
    ids = employee_ids.split(',')  # Split the Employee IDs by comma
    invalid_ids = [eid.strip() for eid in ids if len(eid.strip()) != 6]  # Collect IDs that are not 6 characters

    if invalid_ids:  # If there are invalid IDs, print them with the row index
        print(f"Row {index}: Invalid Employee IDs: {invalid_ids}")

# Apply the function to the 'Employee ID' column, passing the index as well
df1['Employee ID'].apply(lambda x: print_invalid_employee_ids(x, df1[df1['Employee ID'] == x].index[0]))

Row 704: Invalid Employee IDs: ['1025817']
Row 3581: Invalid Employee IDs: ['1004163']
Row 4043: Invalid Employee IDs: ['95366']
Row 4523: Invalid Employee IDs: ['1028517']
Row 4627: Invalid Employee IDs: ['1014']
Row 5296: Invalid Employee IDs: ['2817']
Row 4043: Invalid Employee IDs: ['95366']
Row 8810: Invalid Employee IDs: ['1028217']
Row 10540: Invalid Employee IDs: ['12493254']
Row 10690: Invalid Employee IDs: ['1028127']
Row 4523: Invalid Employee IDs: ['1028517']


,Employee ID
0,None
1,None
2,None
3,None
4,None
...,...
12426,None
12427,None
12428,None
12429,None


In [ ]:
# Define a list of valid Employee IDs to use for replacement
valid_employee_ids = ['100416', '102817', '200090', '100140']

# Function to replace invalid Employee IDs with random valid IDs
def replace_invalid_employee_ids(employee_ids):
    ids = employee_ids.split(',')  # Split the Employee IDs by comma
    new_ids = []

    for eid in ids:
        cleaned_id = eid.strip()  # Remove whitespace
        if len(cleaned_id) != 6:  # Check if the ID is invalid
            # Replace with a random valid ID
            new_id = random.choice(valid_employee_ids)
            new_ids.append(new_id)
        else:
            new_ids.append(cleaned_id)  # Keep the valid ID

    return ', '.join(new_ids)  # Join the IDs back into a string

# Apply the function to the 'Employee ID' column
df1['Employee ID'] = df1['Employee ID'].apply(replace_invalid_employee_ids)

df1.loc[156]

,156
Added Time,2023-07-08 06:32:40
Employee ID,102817
Lot ID,12047602
1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,0
Short - Qty Reject Photoprint 083 OL_PP_DEV,0
2. Scatches Qty Repair,0
Scratches Qty Reject - Photoprint 483_OL_PP_DEV,0
Scratches Qty Reject CHEM B 483 OL_CM_STRP,0
Scratches Qty Reject CHEM C 483 OL_CM_OSP,0
3. Cu Residue Qty Repair,0


In [ ]:
# Function to check for invalid Employee IDs and print them with their row index
def print_invalid_employee_ids(employee_ids, index):
    ids = employee_ids.split(',')  # Split the Employee IDs by comma
    invalid_ids = [eid.strip() for eid in ids if len(eid.strip()) != 6]  # Collect IDs that are not 6 characters

    if invalid_ids:  # If there are invalid IDs, print them with the row index
        print(f"Row {index}: Invalid Employee IDs: {invalid_ids}")

# Apply the function to the 'Employee ID' column, passing the index as well
df1['Employee ID'].apply(lambda x: print_invalid_employee_ids(x, df1[df1['Employee ID'] == x].index[0]))

,Employee ID
0,None
1,None
2,None
3,None
4,None
...,...
12426,None
12427,None
12428,None
12429,None


In [ ]:
# Ensure column is string to allow str.replace
df1['High'] = df1['High'].astype(str)

# Remove 'Qty:' from the values
df1['High'] = df1['High'].str.replace('QTY:', '', regex=False)

# After removal, handle any 'nan' (created by converting NaN to string) and fill them with 0
df1['High'] = df1['High'].replace('nan', np.nan).fillna(0)

# Finally, convert the cleaned values to integer
df1['High'] = df1['High'].astype(int)

print(df1.loc[9, 'High'])

0


In [ ]:
# Ensure column is string to allow str.replace
df1['Low'] = df1['Low'].astype(str)

# Remove 'Qty:' from the values
df1['Low'] = df1['Low'].str.replace('QTY:', '', regex=False)

# After removal, handle any 'nan' (created by converting NaN to string) and fill them with 0
df1['Low'] = df1['Low'].replace('nan', np.nan).fillna(0)

# Finally, convert the cleaned values to integer
df1['Low'] = df1['Low'].astype(int)

print(df1.loc[9, 'Low'])

0


In [ ]:
# Check data types of each column
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12431 entries, 0 to 12430
Data columns (total 60 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Added Time                                                              12431 non-null  datetime64[ns]
 1   Employee ID                                                             12431 non-null  object        
 2   Lot ID                                                                  12431 non-null  object        
 3   1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair          12431 non-null  int64         
 4   Short - Qty Reject Photoprint 083 OL_PP_DEV                             12431 non-null  int64         
 5   2. Scatches Qty Repair                                                  12431 non-null  int64         
 6   Scratches Qty Reject -

In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df1['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Filter the DataFrame to show only rows with invalid Lot IDs
invalid_lot_ids = df1[mask_invalid_lot_id]

# Display the invalid Lot IDs along with their corresponding rows
print(invalid_lot_ids[['Lot ID']])

              Lot ID
1262     CCN00312880
1443     CCN00312881
1482     12103345(B)
1511     CCN00312879
1575     12114264(B)
...              ...
12266  12566054(B/F)
12279  12609028(B/F)
12340  12554937(B/F)
12372  12586376(B/F)
12400  12609028(B/F)

[70 rows x 1 columns]


In [ ]:
# Create a mask for rows where 'Lot ID' contains '(' and is invalid (i.e., not an 8-digit number)
mask_invalid_lot_id_with_parenthesis = df1['Lot ID'].astype(str).str.contains(r'\(|\)', regex=True)

# Filter the DataFrame to show only rows with invalid Lot IDs containing '('
invalid_lot_ids_with_parenthesis = df1[mask_invalid_lot_id_with_parenthesis]

# Display the invalid Lot IDs
print(invalid_lot_ids_with_parenthesis[['Lot ID']])

              Lot ID
1482     12103345(B)
1575     12114264(B)
1632     12114265(B)
1728     12067500(B)
1771     12140270(B)
...              ...
12266  12566054(B/F)
12279  12609028(B/F)
12340  12554937(B/F)
12372  12586376(B/F)
12400  12609028(B/F)

[65 rows x 1 columns]


In [ ]:
# Use regex to remove the content inside parentheses (e.g., '(B)', '(B/F)', etc.)
df1['Lot ID'] = df1['Lot ID'].str.replace(r'\s*\(.*?\)', '', regex=True)

# Check the updated 'Lot ID' column for the rows with invalid Lot IDs
print(df1.loc[invalid_lot_ids_with_parenthesis.index, ['Lot ID']])

         Lot ID
1482   12103345
1575   12114264
1632   12114265
1728   12067500
1771   12140270
...         ...
12266  12566054
12279  12609028
12340  12554937
12372  12586376
12400  12609028

[65 rows x 1 columns]


In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df1['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Create a mask for rows where 'Lot ID' contains '('
mask_lot_id_with_parenthesis = df1['Lot ID'].astype(str).str.contains(r'\(')

# Combine the masks to find invalid Lot IDs that do NOT contain '('
mask_invalid_lot_id_no_parenthesis = mask_invalid_lot_id & ~mask_lot_id_with_parenthesis

# Filter the DataFrame to show only rows with invalid Lot IDs that do not contain '('
invalid_lot_ids_no_parenthesis = df1[mask_invalid_lot_id_no_parenthesis]

# Display the invalid Lot IDs without '('
print(invalid_lot_ids_no_parenthesis[['Lot ID']])

             Lot ID
1262    CCN00312880
1443    CCN00312881
1511    CCN00312879
9677  2425.12376521
9678  2425.12376524


In [ ]:
# Create a mask for rows where 'Lot ID' starts with 'CCN'
mask_ccn_lot_ids = df1['Lot ID'].str.startswith('CCN')

# Apply the split operation only to rows where the 'Lot ID' starts with 'CCN'
df1.loc[mask_ccn_lot_ids, 'Split ID'] = df1.loc[mask_ccn_lot_ids, 'Lot ID'].str[:3]    # Extract 'CCN' into 'Split ID'
df1.loc[mask_ccn_lot_ids, 'Lot ID'] = df1.loc[mask_ccn_lot_ids, 'Lot ID'].str[3:]      # Keep the rest in 'Lot ID'

# Verify the changes by printing the updated DataFrame
df1.loc[1454]

,1454
Added Time,2023-08-28 18:40:04
Employee ID,100416
Lot ID,12067499
1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,0
Short - Qty Reject Photoprint 083 OL_PP_DEV,0
2. Scatches Qty Repair,0
Scratches Qty Reject - Photoprint 483_OL_PP_DEV,0
Scratches Qty Reject CHEM B 483 OL_CM_STRP,0
Scratches Qty Reject CHEM C 483 OL_CM_OSP,0
3. Cu Residue Qty Repair,0


In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df1['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Create a mask for rows where 'Lot ID' contains '('
mask_lot_id_with_parenthesis = df1['Lot ID'].astype(str).str.contains(r'\(')

# Combine the masks to find invalid Lot IDs that do NOT contain '('
mask_invalid_lot_id_no_parenthesis = mask_invalid_lot_id & ~mask_lot_id_with_parenthesis

# Filter the DataFrame to show only rows with invalid Lot IDs that do not contain '('
invalid_lot_ids_no_parenthesis = df1[mask_invalid_lot_id_no_parenthesis]

# Display the invalid Lot IDs without '(' along with their corresponding rows
print(invalid_lot_ids_no_parenthesis[['Lot ID']])

             Lot ID
9677  2425.12376521
9678  2425.12376524


In [ ]:
df1.loc[[9677, 9678], 'Lot ID'] = df.loc[[9677, 9678], 'Lot ID'].str.replace('2425.', '', regex=False)

In [ ]:
# Create a mask for rows where 'Lot ID' does not match 8-digit numbers
mask_invalid_lot_id = df1['Lot ID'].astype(str).str.match(r'^\d{8}$') == False

# Create a mask for rows where 'Lot ID' contains '('
mask_lot_id_with_parenthesis = df1['Lot ID'].astype(str).str.contains(r'\(')

# Combine the masks to find invalid Lot IDs that do NOT contain '('
mask_invalid_lot_id_no_parenthesis = mask_invalid_lot_id & ~mask_lot_id_with_parenthesis

# Filter the DataFrame to show only rows with invalid Lot IDs that do not contain '('
invalid_lot_ids_no_parenthesis = df1[mask_invalid_lot_id_no_parenthesis]

# Display the invalid Lot IDs without '(' along with their corresponding rows
print(invalid_lot_ids_no_parenthesis[['Lot ID']])

Empty DataFrame
Columns: [Lot ID]
Index: []


In [ ]:
# Function to categorize defects
def categorize_short_defects(row):
    defects = []
    # Check for each defect category
    if row['1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair'] > 0 or row['Short - Qty Reject Photoprint 083 OL_PP_DEV'] > 0:
        defects.append('Short (Plating Under Resist)(Dry Film Flake-Off)')
    if row['2. Scatches Qty Repair'] > 0 or row['Scratches Qty Reject - Photoprint 483_OL_PP_DEV'] > 0 or row['Scratches Qty Reject CHEM B 483 OL_CM_STRP'] > 0 or row['Scratches Qty Reject CHEM C 483 OL_CM_OSP'] > 0:
        defects.append('Scratches')
    if row['3. Cu Residue Qty Repair'] > 0 or row['Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP'] > 0:
        defects.append('Cu Residue')
    if row['4. Under-etch Qty Repair'] > 0 or row['Under-etch Qty Reject CHEM B 076 OL_CM_STRP'] > 0:
        defects.append('Under-etch')
    if row['5. Innerlayer Short - Qty Reject 083 IL_STRIP'] > 0:
        defects.append('Innerlayer Short')
    if row['6. Mis-registration Qty Repair'] > 0 or row['Mis-Registration - Qty Reject 471 OL_LAM'] > 0 or row['Mis-Registration - Qty Reject 469 OL_LAM'] > 0:
        defects.append('Mis-registration')
    if row['7. Hole Shifted Qty Reject 132 OL_DRL'] > 0:
        defects.append('Hole Shifted')
    if row['8. Short (Micro-short) Qty Repair'] > 0 or row['Short (Micro-short) Qty Reject CHEM B 083 OL_CM_STRP'] > 0 or row['Short (Micro-short) Qty Reject LPSM 083 OL_SM_CURE'] > 0:
        defects.append('Short (Micro-short)')
    if row['9. Incomplete Solder Strip (Solder Short) Qty Repair'] > 0 or row['Incomplete Solder Strip (Solder Short) Qty Reject CHEM C 256 OL_CM_OSP'] > 0:
        defects.append('Incomplete Solder Strip (Solder Short)')
    if row['10. Feathering (AU Bridging) Qty Repair'] > 0 or row['Feathering (AU Bridging) Qty Reject CHEM C 250 OL_CM_OSP'] > 0:
        defects.append('Feathering (AU Bridging)')
    if row['11. Incomplete Resist Strip -Qty Repair'] > 0 or row['Incomplete Resist Strip Qty Reject CHEM B OL_CM_STRP'] > 0:
        defects.append('Incomplete Resist Strip')
    if row['12. NPTH Short Qty Repair'] > 0 or row['NPTH Short Qty Reject 142'] > 0:
        defects.append('NPTH Short')
    if row['13. Short (Others) Qty Repair'] > 0 or row['Short (Others) Qty Reject'] > 0:
        defects.append('Short (Others)')
    if row['14. Pit & Dent (Others) Qty Reject 479 OL_LAM'] > 0 or row['Pit & Dent (Others) Qty Reject 479 Ol_PP_DEV'] > 0:
        defects.append('Pit & Dent (Others)')
    if row['15. Foil Wrinkle ( Others ) Qty Reject 32 OL_LAM'] > 0:
        defects.append('Foil Wrinkle (Others)')

    return ','.join(defects) if defects else ''

# Function to categorize open defects
def categorize_open_defects(row):
    defects = []
    # Check for each defect category
    if row['1. Innerlayer Open Qty Reject 080 IL_STRIP'] > 0:
        defects.append('Innerlayer Open')
    if row['2. Void Hole Qty Reject CHEM A 180 OL_CM_DM_O'] > 0:
        defects.append('Void Hole')
    if row['3. Partial Feature (Blind Via Void) Qty Reject 477 OL_CM_DM_O'] > 0:
        defects.append('Partial Feature (Blind Via Void)')
    if row['4. Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_LPSM_CURE'] > 0 or row['Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_CM_OSP'] > 0 or row['Lifted Feature (Circuit Peel Off) Qty Reject 473  OL_CM_STRP'] > 0:
        defects.append('Lifted Feature(Circuit Peel-Off)')
    if row['5. Missing Hole Qty Reject 133 OL_DRL'] > 0:
        defects.append('Missing Hole')
    if row['6. Outerlayer Open Qty Repair'] > 0 or row['Outerlayer Open Qty Reject'] > 0:
        defects.append('Outerlayer Open')
    if row['7. Mask on Pad / Finger Qty Repair'] > 0 or row['Mask On Pad/Finger Qty Reject 205 OL_SM_CURE'] > 0:
        defects.append('Mask on Pad/Finger')
    if row['8. Plating Nodule (Particle in Hole)'] > 0 or row['Plating Nodule (Particle In Hole) Qty Reject 178 OL_CM_STRP'] > 0:
        defects.append('Plating Nodule (Particle in Hole)')
    if row['9. Legend On Pad /Hole Qty Repair'] > 0 or row['Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE'] > 0:
        defects.append('Legend on Pad/Hole')
    if row['10. Others Open Qty Repair'] > 0 or row['Others Open Qty Reject'] > 0:
        defects.append('Others Open')
    if row['11. False o/c Qty Repair'] > 0:
        defects.append('False o/c')

    return ','.join(defects) if defects else ''

# Create new columns based on the categorization
df1['Short Defects'] = df1.apply(categorize_short_defects, axis=1)
df1['Open Defects'] = df1.apply(categorize_open_defects, axis=1)

df1.loc[41]

,41
Added Time,2023-07-02 13:39:00
Employee ID,100416
Lot ID,12031426
1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,0
Short - Qty Reject Photoprint 083 OL_PP_DEV,4
...,...
Full Lot ID,12031426
Standardized Date,2023-07-02 00:00:00
Split ID,NaN
Short Defects,Short (Plating Under Resist)(Dry Film Flake-Of...


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12431 entries, 0 to 12430
Data columns (total 62 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Added Time                                                              12431 non-null  datetime64[ns]
 1   Employee ID                                                             12431 non-null  object        
 2   Lot ID                                                                  12431 non-null  object        
 3   1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair          12431 non-null  int64         
 4   Short - Qty Reject Photoprint 083 OL_PP_DEV                             12431 non-null  int64         
 5   2. Scatches Qty Repair                                                  12431 non-null  int64         
 6   Scratches Qty Reject -

In [ ]:
# Function to concatenate columns while ignoring NaN values
def combine_lot_ids(row):
    parts = [row['Split ID'], row['Lot ID'], row['Lot ID Suffix']]
    # Filter out None/NaN values and join the remaining parts with a separator
    return ''.join(str(part) for part in parts if pd.notna(part))

# Apply the function to create the 'Full Lot ID'
df1['Full Lot ID'] = df1.apply(combine_lot_ids, axis=1)

# Display the DataFrame with the new 'Full Lot ID'
print(df1[df1['Lot ID'] == '12174606'])

              Added Time Employee ID    Lot ID  \
1824 2023-09-14 12:30:28      100416  12174606   

      1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair  \
1824                                                  0                

      Short - Qty Reject Photoprint 083 OL_PP_DEV  2. Scatches Qty Repair  \
1824                                            0                       0   

      Scratches Qty Reject - Photoprint 483_OL_PP_DEV  \
1824                                                0   

      Scratches Qty Reject CHEM B 483 OL_CM_STRP  \
1824                                           0   

      Scratches Qty Reject CHEM C 483 OL_CM_OSP  3. Cu Residue Qty Repair  \
1824                                          0                         0   

      ...  Impedance Fail (Y/N)  High  Low  Final Passed Qty  Lot ID Suffix  \
1824  ...                    NO     0    0                20        (4WIRE)   

          Full Lot ID  Standardized Date  Split ID  Short Defects

In [ ]:
# Check duplicates in your main dataframe
duplicated_lot_nos = df1[df1.duplicated(subset='Full Lot ID', keep=False)]['Full Lot ID'].unique()
print(duplicated_lot_nos)
print("Total number of duplicated 'Lot No.':", len(duplicated_lot_nos))

['12037077(680)' '12078158' '12103345(4WIRE)' '12067484(4WIRE)'
 '12067499(4WIRE)' '12067500(4WIRE)' '12104050' '12140270(4WIRE)'
 '12170333(4WIRE)' '12034374' '12127491' '12150830(4WIRE)'
 '12113172(4WIRE)' '12119211(4WIRE)' '12103424(4WIRE)' '12135071'
 '12098221(4WIRE)' '12131997(4WIRE)' '12140284(4WIRE)' '12144295(4WIRE)'
 '12144241(4WIRE)' '12144239(4WIRE)' '12144240(4WIRE)' '12193099(4WIRE)'
 '12193099' '12191975' '12227558(4WIRE)' '12219341' '12199455' '12199448'
 '12170754' '12196377' '12130510' '12232322' '12123122' '12251697(4WIRE)'
 '12212456' '12265428(4WIRE)' '12274834' '12264151' '12264536' '12294593'
 '12256857' '12253541' '12236134' '12287651' '12259387' '12278105'
 '12278109' '12250729' '12250727' '12316374' '12341337' '12316524'
 '12328472' '12313641' '12355351(680, TEST 1)' '12373089(4WIRE)'
 '12373113(4WIRE)' '12355347(4WIRE)' '12380069(4WIRE)' '12378115'
 '12372999(4WIRE)' '12411881' '12424937(4WIRE)' '12404979'
 '12404979(4WIRE)' '12406453(4WIRE)' '12433528' '1231

In [ ]:
# Specify columns to ignore when checking for duplicates
cols_to_ignore = ['Added Time', 'Standardized Date', 'Employee ID']

# Create a list of columns to consider for dropping duplicates (all except the ignored ones)
cols_to_check = df1.columns.difference(cols_to_ignore)

# Drop duplicates based on the specified columns
df1_cleaned = df1.drop_duplicates(subset=cols_to_check, keep='last')

# Optional: Reset index if needed
df1_cleaned.reset_index(drop=True, inplace=True)

# Print the cleaned DataFrame
df1_cleaned

,Added Time,Employee ID,Lot ID,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,Scratches Qty Reject CHEM B 483 OL_CM_STRP,Scratches Qty Reject CHEM C 483 OL_CM_OSP,3. Cu Residue Qty Repair,...,Impedance Fail (Y/N),High,Low,Final Passed Qty,Lot ID Suffix,Full Lot ID,Standardized Date,Split ID,Short Defects,Open Defects
0,2022-01-01 01:26:53,102817,11487783,0,0,0,0,0,0,0,...,NO,0,0,3800,NaN,11487783,2022-01-01,NaN,,Innerlayer Open
1,2023-07-01 03:17:08,102817,12105804,0,0,0,0,0,0,0,...,NO,0,0,91,NaN,12105804,2023-07-01,NaN,,
2,2023-07-01 03:17:36,102817,12107143,0,0,0,0,0,0,0,...,NO,0,0,42,NaN,12107143,2023-07-01,NaN,,
3,2023-07-01 03:19:43,102817,12107226,0,0,0,0,0,0,0,...,NO,0,0,322,NaN,12107226,2023-07-01,NaN,,Innerlayer Open
4,2023-07-01 03:21:45,102817,12111643,0,0,0,0,0,0,0,...,NO,0,0,84,NaN,12111643,2023-07-01,NaN,Innerlayer Short,Innerlayer Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12390,2025-01-08 13:10:12,100140,12593102,0,0,0,0,0,0,0,...,NO,0,0,1108,NaN,12593102,2025-01-08,NaN,,Plating Nodule (Particle in Hole)
12391,2025-01-08 15:30:28,100140,12587392,0,0,0,0,0,0,0,...,NO,0,0,510,(SILVER),12587392(SILVER),2025-01-08,NaN,,Others Open
12392,2025-01-08 16:01:33,100140,12587705,0,0,0,0,0,0,0,...,NO,0,0,2753,NaN,12587705,2025-01-08,NaN,,Mask on Pad/Finger
12393,2025-01-08 16:12:59,100140,12588590,0,0,0,0,0,0,0,...,NO,0,0,88,(SILVER),12588590(SILVER),2025-01-08,NaN,,Others Open


In [ ]:
# Check duplicates in your main dataframe
duplicated_lot_nos = df1_cleaned[df1_cleaned.duplicated(subset='Full Lot ID', keep=False)]['Full Lot ID'].unique()
print(duplicated_lot_nos)
print("Total number of duplicated 'Lot No.':", len(duplicated_lot_nos))

['12103345(4WIRE)' '12067484(4WIRE)' '12067499(4WIRE)' '12067500(4WIRE)'
 '12140270(4WIRE)' '12170333(4WIRE)' '12127491' '12150830(4WIRE)'
 '12113172(4WIRE)' '12119211(4WIRE)' '12103424(4WIRE)' '12098221(4WIRE)'
 '12131997(4WIRE)' '12140284(4WIRE)' '12144295(4WIRE)' '12144241(4WIRE)'
 '12144239(4WIRE)' '12144240(4WIRE)' '12193099(4WIRE)' '12193099'
 '12191975' '12232322' '12265428(4WIRE)' '12274834' '12264151' '12256857'
 '12253541' '12236134' '12278105' '12278109' '12250729' '12373089(4WIRE)'
 '12373113(4WIRE)' '12380069(4WIRE)' '12378115' '12372999(4WIRE)'
 '12411881' '12424937(4WIRE)' '12404979' '12404979(4WIRE)'
 '12406453(4WIRE)' '12433528' '12316593(SILVER)' '12465335(4WIRE)'
 '12443748' '12489642' '12492320(4WIRE)' '12422742(4WIRE)'
 '12498736(4WIRE)' '12459501(4WIRE)' '12452792' '12461321' '12487121'
 'B12473797' '12532356(4WIRE)' '12498388(4WIRE)' '12532355(4WIRE)'
 '12554999(4WIRE)' '12554938(4WIRE)' '12554937(4WIRE)' '12554998(4WIRE)'
 '12555014(4WIRE)' '12555000(4WIRE)' '12

In [ ]:
# Sort by 'Added Time' in ascending order to keep the latest entry
df1_sorted = df1_cleaned.sort_values(by='Added Time', ascending=False)

# Drop duplicates based on 'Full Lot No', keeping only the latest occurrence
df1_cleaned = df1_sorted.drop_duplicates(subset='Full Lot ID')

# Sort the DataFrame again based on 'Added Time' in ascending order
df1_cleaned = df1_cleaned.sort_values(by='Added Time', ascending=True)

# Optional: Reset index if needed
df1_cleaned.reset_index(drop=True, inplace=True)

# Check the cleaned DataFrame
df1_cleaned.head()

,Added Time,Employee ID,Lot ID,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,Scratches Qty Reject CHEM B 483 OL_CM_STRP,Scratches Qty Reject CHEM C 483 OL_CM_OSP,3. Cu Residue Qty Repair,...,Impedance Fail (Y/N),High,Low,Final Passed Qty,Lot ID Suffix,Full Lot ID,Standardized Date,Split ID,Short Defects,Open Defects
0,2022-01-01 01:26:53,102817,11487783,0,0,0,0,0,0,0,...,NO,0,0,3800,NaN,11487783,2022-01-01,NaN,,Innerlayer Open
1,2023-07-01 03:17:08,102817,12105804,0,0,0,0,0,0,0,...,NO,0,0,91,NaN,12105804,2023-07-01,NaN,,
2,2023-07-01 03:17:36,102817,12107143,0,0,0,0,0,0,0,...,NO,0,0,42,NaN,12107143,2023-07-01,NaN,,
3,2023-07-01 03:19:43,102817,12107226,0,0,0,0,0,0,0,...,NO,0,0,322,NaN,12107226,2023-07-01,NaN,,Innerlayer Open
4,2023-07-01 03:21:45,102817,12111643,0,0,0,0,0,0,0,...,NO,0,0,84,NaN,12111643,2023-07-01,NaN,Innerlayer Short,Innerlayer Open


In [ ]:
duplicated_lot_nos = df1_cleaned[df1_cleaned.duplicated(subset='Full Lot ID', keep=False)]['Full Lot ID'].unique()
print(duplicated_lot_nos)
print("Total number of duplicated 'Lot No.':", len(duplicated_lot_nos))

[]
Total number of duplicated 'Lot No.': 0


In [ ]:
df1_cleaned['Lot ID'] = df1_cleaned['Lot ID'].astype(str)
df1_cleaned['Split ID'] = df1_cleaned['Split ID'].astype(str)
df1_cleaned['Lot ID Suffix'] = df1_cleaned['Lot ID Suffix'].astype(str)

In [ ]:
# Print the number of rows before and after dropping duplicates
print(f"Original DataFrame shape: {df1.shape}")
print(f"Cleaned DataFrame shape: {df1_cleaned.shape}")

Original DataFrame shape: (12431, 62)
Cleaned DataFrame shape: (12310, 62)


In [ ]:
selected_cols = [
    'Standardized Date',
    'Employee ID',
    'Split ID',
    'Lot ID',
    'Lot ID Suffix',
    'Short Defects',
    '1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair',
    'Short - Qty Reject Photoprint 083 OL_PP_DEV',
    '2. Scatches Qty Repair',
    'Scratches Qty Reject - Photoprint 483_OL_PP_DEV',
    'Scratches Qty Reject CHEM B 483 OL_CM_STRP',
    'Scratches Qty Reject CHEM C 483 OL_CM_OSP',
    '3. Cu Residue Qty Repair',
    'Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP',
    '4. Under-etch Qty Repair',
    'Under-etch Qty Reject CHEM B 076 OL_CM_STRP',
    '5. Innerlayer Short - Qty Reject 083 IL_STRIP',
    '6. Mis-registration Qty Repair',
    'Mis-Registration - Qty Reject 471 OL_LAM',
    'Mis-Registration - Qty Reject 469 OL_LAM',
    '7. Hole Shifted Qty Reject 132 OL_DRL',
    '8. Short (Micro-short) Qty Repair',
    'Short (Micro-short) Qty Reject CHEM B 083 OL_CM_STRP',
    'Short (Micro-short) Qty Reject LPSM 083 OL_SM_CURE',
    '9. Incomplete Solder Strip (Solder Short) Qty Repair',
    'Incomplete Solder Strip (Solder Short) Qty Reject CHEM C 256 OL_CM_OSP',
    '10. Feathering (AU Bridging) Qty Repair',
    'Feathering (AU Bridging) Qty Reject CHEM C 250 OL_CM_OSP',
    '11. Incomplete Resist Strip -Qty Repair',
    'Incomplete Resist Strip Qty Reject CHEM B OL_CM_STRP',
    '12. NPTH Short Qty Repair',
    'NPTH Short Qty Reject 142',
    '13. Short (Others) Qty Repair',
    'Short (Others) Qty Reject',
    '14. Pit & Dent (Others) Qty Reject 479 OL_LAM',
    'Pit & Dent (Others) Qty Reject 479 Ol_PP_DEV',
    '15. Foil Wrinkle ( Others ) Qty Reject 32 OL_LAM',
    'Open Defects',
    '1. Innerlayer Open Qty Reject 080 IL_STRIP',
    '2. Void Hole Qty Reject CHEM A 180 OL_CM_DM_O',
    '3. Partial Feature (Blind Via Void) Qty Reject 477 OL_CM_DM_O',
    '4. Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_LPSM_CURE',
    'Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_CM_OSP',
    'Lifted Feature (Circuit Peel Off) Qty Reject 473  OL_CM_STRP',
    '5. Missing Hole Qty Reject 133 OL_DRL',
    '6. Outerlayer Open Qty Repair',
    'Outerlayer Open Qty Reject',
    '7. Mask on Pad / Finger Qty Repair',
    'Mask On Pad/Finger Qty Reject 205 OL_SM_CURE',
    '8. Plating Nodule (Particle in Hole)',
    'Plating Nodule (Particle In Hole) Qty Reject 178 OL_CM_STRP',
    '9. Legend On Pad /Hole Qty Repair',
    'Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE',
    '10. Others Open Qty Repair',
    'Others Open Qty Reject',
    '11. False o/c Qty Repair',
    'Impedance Fail (Y/N)',
    'High',
    'Low',
    'Final Passed Qty',
    'Full Lot ID'
]

new_df1 = df1_cleaned[selected_cols]

new_df1.head()

,Standardized Date,Employee ID,Split ID,Lot ID,Lot ID Suffix,Short Defects,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,...,9. Legend On Pad /Hole Qty Repair,Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE,10. Others Open Qty Repair,Others Open Qty Reject,11. False o/c Qty Repair,Impedance Fail (Y/N),High,Low,Final Passed Qty,Full Lot ID
0,2022-01-01,102817,nan,11487783,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,3800,11487783
1,2023-07-01,102817,nan,12105804,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,91,12105804
2,2023-07-01,102817,nan,12107143,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,42,12107143
3,2023-07-01,102817,nan,12107226,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,322,12107226
4,2023-07-01,102817,nan,12111643,nan,Innerlayer Short,0,0,0,0,...,0,0,0,0,0,NO,0,0,84,12111643


In [ ]:
new_df1['Employee ID'] = new_df1['Employee ID'].astype(str)

<ipython-input-91-3786a2370c1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df1['Employee ID'] = new_df1['Employee ID'].astype(str)


In [ ]:
new_df1.dtypes

,0
Standardized Date,datetime64[ns]
Employee ID,object
Split ID,object
Lot ID,object
Lot ID Suffix,object
...,...
Impedance Fail (Y/N),object
High,int64
Low,int64
Final Passed Qty,int64


# **combined_df**

In [ ]:
new_df

,Standardized Date,Done By,Split ID,Lot ID,Tool Number (S. No.),Kevin Probe Test,For 4Wire,Date Code,Date Code Input,Job Type,...,Isolation,Adjacency Used ( Emma Only),Lot ID Suffix,Mfg Qty,Tested Qty,1st Passed Qty,Open,Short,SDP,Full Lot ID
0,2023-01-01,102480,NaN,11387972,S03804_A,NO,NaN,Yes,2252,STANDARD,...,10,NO,NaN,630,592,465,63,64,38,11387972
1,2023-07-01,101754,NaN,12105804,S11664_E,NO,NaN,Yes,2623,QTA,...,10,NO,NaN,120,91,91,0,0,29,12105804
2,2023-07-01,102477,NaN,12107143,S12881_B,NO,NaN,Yes,2623,NPI,...,1048,YES,NaN,48,42,42,0,0,6,12107143
3,2023-07-01,102477,NaN,12107226,S12949_A,NO,NaN,Yes,2523,QTA,...,10,YES,NaN,336,326,322,4,0,10,12107226
4,2023-07-01,101754,NaN,12073207,S11515_J,NO,NaN,Yes,2623,STANDARD,...,10,NO,NaN,432,376,336,40,0,56,12073207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,2025-01-08,200090,NaN,12587705,S13203_A,NO,NaN,Yes,0225,STANDARD,...,10,NO,NaN,2760,2755,2753,1,1,5,12587705
12405,2025-01-08,200090,NaN,12587392,S12453_A,NO,NaN,Yes,2452,STANDARD,...,10,NO,(SILVER),540,513,510,1,2,27,12587392(SILVER)
12406,2025-01-08,101664,NaN,12588590,S12695_B,NO,NaN,Yes,0225,QTA,...,10,YES,(SILVER),120,89,88,1,0,31,12588590(SILVER)
12407,2025-01-08,102415,NaN,12591568,S09018_C,NO,NaN,Yes,0225,STANDARD,...,10,NO,NaN,84,79,78,1,0,5,12591568


In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12409 entries, 0 to 12408
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Standardized Date            12409 non-null  datetime64[ns]
 1   Done By                      12409 non-null  object        
 2   Split ID                     477 non-null    object        
 3   Lot ID                       12409 non-null  object        
 4   Tool Number (S. No.)         12409 non-null  object        
 5   Kevin Probe Test             12409 non-null  object        
 6   For 4Wire                    955 non-null    object        
 7   Date Code                    12409 non-null  object        
 8   Date Code Input              12409 non-null  object        
 9   Job Type                     12409 non-null  object        
 10  Machine                      12409 non-null  object        
 11  X-Cut Allowed                12409 non-nu

In [ ]:
new_df1.head()

,Standardized Date,Employee ID,Split ID,Lot ID,Lot ID Suffix,Short Defects,1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair,Short - Qty Reject Photoprint 083 OL_PP_DEV,2. Scatches Qty Repair,Scratches Qty Reject - Photoprint 483_OL_PP_DEV,...,9. Legend On Pad /Hole Qty Repair,Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE,10. Others Open Qty Repair,Others Open Qty Reject,11. False o/c Qty Repair,Impedance Fail (Y/N),High,Low,Final Passed Qty,Full Lot ID
0,2022-01-01,102817,nan,11487783,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,3800,11487783
1,2023-07-01,102817,nan,12105804,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,91,12105804
2,2023-07-01,102817,nan,12107143,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,42,12107143
3,2023-07-01,102817,nan,12107226,nan,,0,0,0,0,...,0,0,0,0,0,NO,0,0,322,12107226
4,2023-07-01,102817,nan,12111643,nan,Innerlayer Short,0,0,0,0,...,0,0,0,0,0,NO,0,0,84,12111643


In [ ]:
new_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12310 entries, 0 to 12309
Data columns (total 61 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Standardized Date                                                       12310 non-null  datetime64[ns]
 1   Employee ID                                                             12310 non-null  object        
 2   Split ID                                                                12310 non-null  object        
 3   Lot ID                                                                  12310 non-null  object        
 4   Lot ID Suffix                                                           12310 non-null  object        
 5   Short Defects                                                           12310 non-null  object        
 6   1. Short (Plating Unde

In [ ]:
df_combined = pd.merge(new_df, new_df1, on='Full Lot ID', how='left')
df_combined

,Standardized Date_x,Done By,Split ID_x,Lot ID_x,Tool Number (S. No.),Kevin Probe Test,For 4Wire,Date Code,Date Code Input,Job Type,...,Plating Nodule (Particle In Hole) Qty Reject 178 OL_CM_STRP,9. Legend On Pad /Hole Qty Repair,Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE,10. Others Open Qty Repair,Others Open Qty Reject,11. False o/c Qty Repair,Impedance Fail (Y/N),High,Low,Final Passed Qty
0,2023-01-01,102480,NaN,11387972,S03804_A,NO,NaN,Yes,2252,STANDARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-07-01,101754,NaN,12105804,S11664_E,NO,NaN,Yes,2623,QTA,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,91.0
2,2023-07-01,102477,NaN,12107143,S12881_B,NO,NaN,Yes,2623,NPI,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,42.0
3,2023-07-01,102477,NaN,12107226,S12949_A,NO,NaN,Yes,2523,QTA,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,322.0
4,2023-07-01,101754,NaN,12073207,S11515_J,NO,NaN,Yes,2623,STANDARD,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,339.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,2025-01-08,200090,NaN,12587705,S13203_A,NO,NaN,Yes,0225,STANDARD,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,2753.0
12405,2025-01-08,200090,NaN,12587392,S12453_A,NO,NaN,Yes,2452,STANDARD,...,0.0,0.0,0.0,0.0,3.0,0.0,NO,0.0,0.0,510.0
12406,2025-01-08,101664,NaN,12588590,S12695_B,NO,NaN,Yes,0225,QTA,...,0.0,0.0,0.0,0.0,1.0,0.0,NO,0.0,0.0,88.0
12407,2025-01-08,102415,NaN,12591568,S09018_C,NO,NaN,Yes,0225,STANDARD,...,0.0,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,78.0


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12409 entries, 0 to 12408
Data columns (total 87 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Standardized Date_x                                                     12409 non-null  datetime64[ns]
 1   Done By                                                                 12409 non-null  object        
 2   Split ID_x                                                              477 non-null    object        
 3   Lot ID_x                                                                12409 non-null  object        
 4   Tool Number (S. No.)                                                    12409 non-null  object        
 5   Kevin Probe Test                                                        12409 non-null  object        
 6   For 4Wire             

In [ ]:
# Identify duplicated rows in df_combined
duplicated_rows = df_combined[df_combined.duplicated(keep=False)]

# Print the duplicated rows
print("Duplicated Rows in df_combined:")
print(duplicated_rows)

Duplicated Rows in df_combined:
Empty DataFrame
Columns: [Standardized Date_x, Done By, Split ID_x, Lot ID_x, Tool Number (S. No.), Kevin Probe Test, For 4Wire, Date Code, Date Code Input, Job Type, Machine, X-Cut Allowed, IPC Class, Mass Lam, Resistive, Continuity, Test Voltage, Isolation, Adjacency Used ( Emma Only), Lot ID Suffix_x, Mfg Qty, Tested Qty, 1st Passed Qty, Open, Short, SDP, Full Lot ID, Standardized Date_y, Employee ID, Split ID_y, Lot ID_y, Lot ID Suffix_y, Short Defects, 1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair, Short - Qty Reject Photoprint 083 OL_PP_DEV, 2. Scatches Qty Repair, Scratches Qty Reject - Photoprint 483_OL_PP_DEV, Scratches Qty Reject CHEM B 483 OL_CM_STRP, Scratches Qty Reject CHEM C 483 OL_CM_OSP, 3. Cu Residue Qty Repair, Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP, 4. Under-etch Qty Repair, Under-etch Qty Reject CHEM B 076 OL_CM_STRP, 5. Innerlayer Short - Qty Reject 083 IL_STRIP, 6. Mis-registration Qty Repair, Mis-Registrati

In [ ]:
# Check how many rows have NaT in the 'Standardized Date_x' column
nat_count = df_combined['Standardized Date_x'].isna().sum()

# Output the result
print(f"Number of rows with NaT in 'Standardized Date_x': {nat_count}")

Number of rows with NaT in 'Standardized Date_x': 0


In [ ]:
# Combine values from 'Standardized Date_x' and 'Standardized Date_y', preferring non-null values
df_combined['Standardized Date'] = df_combined['Standardized Date_x'].combine_first(df_combined['Standardized Date_y'])

# Drop the original columns if they are no longer needed
df_combined.drop(columns=['Standardized Date_x', 'Standardized Date_y'], inplace=True)

In [ ]:
df_combined['Lot ID'] = df_combined['Lot ID_x'].fillna('') + ', ' + df_combined['Lot ID_y'].fillna('')

# If both columns are the same, remove the duplication
df_combined['Lot ID'] = df_combined['Lot ID_x'].where(df_combined['Lot ID_x'] != df_combined['Lot ID_y'], df_combined['Lot ID_x'])

# Remove any leading or trailing commas or spaces
df_combined['Lot ID'] = df_combined['Lot ID'].str.strip(', ')

# Drop the original columns if they are no longer needed
df_combined.drop(columns=['Lot ID_x', 'Lot ID_y'], inplace=True)

In [ ]:
df_combined['Lot ID Suffix'] = df_combined['Lot ID Suffix_x'].fillna('').astype(str) + ', ' + df_combined['Lot ID Suffix_y'].fillna('')

# If both columns are the same, remove the duplication
df_combined['Lot ID Suffix'] = df_combined['Lot ID Suffix_x'].where(df_combined['Lot ID Suffix_x'] != df_combined['Lot ID Suffix_y'], df_combined['Lot ID Suffix_x'])

# Remove any leading or trailing commas or spaces
df_combined['Lot ID Suffix'] = df_combined['Lot ID Suffix'].str.strip(', ')

# Drop the original columns if they are no longer needed
df_combined.drop(columns=['Lot ID Suffix_x', 'Lot ID Suffix_y'], inplace=True)

In [ ]:
df_combined['Split ID'] = df_combined['Split ID_x'].fillna('') + ', ' + df_combined['Split ID_y'].fillna('')

# If both columns are the same, remove the duplication
df_combined['Split ID'] = df_combined['Split ID_x'].where(df_combined['Split ID_x'] != df_combined['Split ID_y'], df_combined['Split ID_x'])

# Remove any leading or trailing commas or spaces
df_combined['Split ID'] = df_combined['Split ID'].str.strip(', ')

# Drop the original columns if they are no longer needed
df_combined.drop(columns=['Split ID_x', 'Split ID_y'], inplace=True)

In [ ]:
df_combined.rename(columns={
    'Done By': 'Done By - Testing',
    'Employee ID': 'Done By - Tracing'
}, inplace=True)

In [ ]:
df_combined

,Done By - Testing,Tool Number (S. No.),Kevin Probe Test,For 4Wire,Date Code,Date Code Input,Job Type,Machine,X-Cut Allowed,IPC Class,...,Others Open Qty Reject,11. False o/c Qty Repair,Impedance Fail (Y/N),High,Low,Final Passed Qty,Standardized Date,Lot ID,Lot ID Suffix,Split ID
0,102480,S03804_A,NO,NaN,Yes,2252,STANDARD,VERSA,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01,11387972,NaN,NaN
1,101754,S11664_E,NO,NaN,Yes,2623,QTA,KAIMA,0.0,2.0,...,0.0,0.0,NO,0.0,0.0,91.0,2023-07-01,12105804,NaN,NaN
2,102477,S12881_B,NO,NaN,Yes,2623,NPI,6151 (1),1.0,2.0,...,0.0,0.0,NO,0.0,0.0,42.0,2023-07-01,12107143,NaN,NaN
3,102477,S12949_A,NO,NaN,Yes,2523,QTA,4033,0.0,2.0,...,0.0,0.0,NO,0.0,0.0,322.0,2023-07-01,12107226,NaN,NaN
4,101754,S11515_J,NO,NaN,Yes,2623,STANDARD,KAIMA,0.0,2.0,...,0.0,0.0,NO,0.0,0.0,339.0,2023-07-01,12073207,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,200090,S13203_A,NO,NaN,Yes,0225,STANDARD,KAIMA,1.0,2.0,...,0.0,0.0,NO,0.0,0.0,2753.0,2025-01-08,12587705,NaN,NaN
12405,200090,S12453_A,NO,NaN,Yes,2452,STANDARD,KAIMA,0.0,3.0,...,3.0,0.0,NO,0.0,0.0,510.0,2025-01-08,12587392,(SILVER),NaN
12406,101664,S12695_B,NO,NaN,Yes,0225,QTA,6151 (2),0.0,2.0,...,1.0,0.0,NO,0.0,0.0,88.0,2025-01-08,12588590,(SILVER),NaN
12407,102415,S09018_C,NO,NaN,Yes,0225,STANDARD,LM1,0.0,2.0,...,0.0,0.0,NO,0.0,0.0,78.0,2025-01-08,12591568,NaN,NaN


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12409 entries, 0 to 12408
Data columns (total 83 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Done By - Testing                                                       12409 non-null  object        
 1   Tool Number (S. No.)                                                    12409 non-null  object        
 2   Kevin Probe Test                                                        12409 non-null  object        
 3   For 4Wire                                                               955 non-null    object        
 4   Date Code                                                               12409 non-null  object        
 5   Date Code Input                                                         12409 non-null  object        
 6   Job Type              

In [ ]:
new_order = ['Standardized Date', 'Done By - Testing', 'Done By - Tracing', 'Split ID', 'Lot ID', 'Lot ID Suffix', 'Tool Number (S. No.)', 'Kevin Probe Test',
             'For 4Wire', 'Date Code', 'Date Code Input', 'Job Type', 'Machine', 'X-Cut Allowed', 'IPC Class',
             'Mass Lam', 'Resistive', 'Continuity', 'Test Voltage', 'Isolation', 'Adjacency Used ( Emma Only)',
             'Mfg Qty', 'Tested Qty', '1st Passed Qty', 'Open', 'Short', 'SDP', 'Open Defects',
             '1. Innerlayer Open Qty Reject 080 IL_STRIP', '2. Void Hole Qty Reject CHEM A 180 OL_CM_DM_O', '3. Partial Feature (Blind Via Void) Qty Reject 477 OL_CM_DM_O',
             '4. Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_LPSM_CURE', 'Lifted Feature (Circuit Peel Off) Qty Reject 473 OL_CM_OSP', 'Lifted Feature (Circuit Peel Off) Qty Reject 473  OL_CM_STRP',
             '5. Missing Hole Qty Reject 133 OL_DRL', '6. Outerlayer Open Qty Repair', 'Outerlayer Open Qty Reject', '7. Mask on Pad / Finger Qty Repair', 'Mask On Pad/Finger Qty Reject 205 OL_SM_CURE', '8. Plating Nodule (Particle in Hole)',
             'Plating Nodule (Particle In Hole) Qty Reject 178 OL_CM_STRP', '9. Legend On Pad /Hole Qty Repair', 'Legend On Pad/Hole Qty Reject 233 OL_LPSM_CURE', '10. Others Open Qty Repair', 'Others Open Qty Reject',
             '11. False o/c Qty Repair', 'Short Defects', '1. Short (Plating Under Resist)(Dry Film Flake-Off) Qty Repair', 'Short - Qty Reject Photoprint 083 OL_PP_DEV', '2. Scatches Qty Repair',
             'Scratches Qty Reject - Photoprint 483_OL_PP_DEV', 'Scratches Qty Reject CHEM B 483 OL_CM_STRP', 'Scratches Qty Reject CHEM C 483 OL_CM_OSP', '3. Cu Residue Qty Repair', 'Cu Residue Qty Reject - CHEM B 172 OL_CM_STRP',
             '4. Under-etch Qty Repair', 'Under-etch Qty Reject CHEM B 076 OL_CM_STRP', '5. Innerlayer Short - Qty Reject 083 IL_STRIP', '6. Mis-registration Qty Repair', 'Mis-Registration - Qty Reject 471 OL_LAM', 'Mis-Registration - Qty Reject 469 OL_LAM',
             '7. Hole Shifted Qty Reject 132 OL_DRL', '8. Short (Micro-short) Qty Repair', 'Short (Micro-short) Qty Reject CHEM B 083 OL_CM_STRP', 'Short (Micro-short) Qty Reject LPSM 083 OL_SM_CURE', '9. Incomplete Solder Strip (Solder Short) Qty Repair',
             'Incomplete Solder Strip (Solder Short) Qty Reject CHEM C 256 OL_CM_OSP', '10. Feathering (AU Bridging) Qty Repair', 'Feathering (AU Bridging) Qty Reject CHEM C 250 OL_CM_OSP', '11. Incomplete Resist Strip -Qty Repair', 'Incomplete Resist Strip Qty Reject CHEM B OL_CM_STRP',
             '12. NPTH Short Qty Repair', 'NPTH Short Qty Reject 142', '13. Short (Others) Qty Repair', 'Short (Others) Qty Reject', '14. Pit & Dent (Others) Qty Reject 479 OL_LAM', 'Pit & Dent (Others) Qty Reject 479 Ol_PP_DEV',
             '15. Foil Wrinkle ( Others ) Qty Reject 32 OL_LAM', 'Impedance Fail (Y/N)', 'High', 'Low', 'Final Passed Qty', 'Full Lot ID']

df_combined = df_combined[new_order]

df_combined

,Standardized Date,Done By - Testing,Done By - Tracing,Split ID,Lot ID,Lot ID Suffix,Tool Number (S. No.),Kevin Probe Test,For 4Wire,Date Code,...,13. Short (Others) Qty Repair,Short (Others) Qty Reject,14. Pit & Dent (Others) Qty Reject 479 OL_LAM,Pit & Dent (Others) Qty Reject 479 Ol_PP_DEV,15. Foil Wrinkle ( Others ) Qty Reject 32 OL_LAM,Impedance Fail (Y/N),High,Low,Final Passed Qty,Full Lot ID
0,2023-01-01,102480,NaN,NaN,11387972,NaN,S03804_A,NO,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11387972
1,2023-07-01,101754,102817,NaN,12105804,NaN,S11664_E,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,91.0,12105804
2,2023-07-01,102477,102817,NaN,12107143,NaN,S12881_B,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,42.0,12107143
3,2023-07-01,102477,102817,NaN,12107226,NaN,S12949_A,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,322.0,12107226
4,2023-07-01,101754,102817,NaN,12073207,NaN,S11515_J,NO,NaN,Yes,...,0.0,1.0,0.0,0.0,0.0,NO,0.0,0.0,339.0,12073207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12404,2025-01-08,200090,100140,NaN,12587705,NaN,S13203_A,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,2753.0,12587705
12405,2025-01-08,200090,100140,NaN,12587392,(SILVER),S12453_A,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,510.0,12587392(SILVER)
12406,2025-01-08,101664,100140,NaN,12588590,(SILVER),S12695_B,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,88.0,12588590(SILVER)
12407,2025-01-08,102415,100140,NaN,12591568,NaN,S09018_C,NO,NaN,Yes,...,0.0,0.0,0.0,0.0,0.0,NO,0.0,0.0,78.0,12591568


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12409 entries, 0 to 12408
Data columns (total 83 columns):
 #   Column                                                                  Non-Null Count  Dtype         
---  ------                                                                  --------------  -----         
 0   Standardized Date                                                       12409 non-null  datetime64[ns]
 1   Done By - Testing                                                       12409 non-null  object        
 2   Done By - Tracing                                                       12303 non-null  object        
 3   Split ID                                                                477 non-null    object        
 4   Lot ID                                                                  12409 non-null  object        
 5   Lot ID Suffix                                                           2080 non-null   object        
 6   Tool Number (S. No.)  

In [ ]:
# If you have an existing Google Sheet, open it by title or URL
spreadsheet = gc.open('Old ET Form Output')
worksheet = spreadsheet.get_worksheet(0)

# Now upload df_combined to the Google Sheet
set_with_dataframe(worksheet, df_combined)

print("DataFrame uploaded successfully to Google Sheets.")

DataFrame uploaded successfully to Google Sheets.
